In [16]:
%load_ext autoreload
%autoreload 2

import os
import random
import sys
import logging
import time
from datetime import datetime

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from pathlib import Path
from hydra import initialize_config_dir, compose
from hydra.utils import instantiate
from omegaconf import OmegaConf
import tqdm
import torch
import matplotlib.pyplot as plt

from tbfm import film
from tbfm import multisession
from tbfm import utils

DATA_DIR = "/home/danmuir/Projects/tbfm_multisession/data"
sys.path.append(DATA_DIR)
from tbfm import dataset
meta = dataset.load_meta(DATA_DIR)

OUT_DIR = "data"
EMBEDDING_REST_SUBDIR = "embedding_rest"

conf_dir = Path("./conf").resolve()

with initialize_config_dir(config_dir=str(conf_dir), version_base=None):
    cfg = compose(config_name="config")

DEVICE = "cuda"
WINDOW_SIZE = cfg.data.trial_len
NUM_HELD_OUT_SESSIONS = cfg.training.num_held_out_sessions

ImportError: cannot import name 'film' from 'tbfm' (/home/danmuir/GitHub/py-tbfm/tbfm/__init__.py)

# Multisession TBFM Training with Multiple Approaches

This notebook trains and compares multiple TBFM model variants:

## Available Models
1. **Baseline**: Standard TBFM with PCA-initialized autoencoder
2. **Spatial Regularization**: Adds spatial smoothness penalties  
3. **AE Reconstruction**: Adds autoencoder reconstruction penalty
4. **Sparsity-Based**: Identity AE + sparsity penalties to identify important channels

## How to Use This Notebook
- **Run all cells sequentially** to train all models
- **Or skip sections** to train only specific models (notebook handles missing data gracefully)
- **Comparison cells automatically detect** which models are trained
- All sections after "Data Loading" are independent except where noted

## Setup Logging

In [ ]:
class ExperimentLogger:
    """Logger that writes to both console and file with timing information."""
    
    def __init__(self, log_dir="logs", experiment_name="multisession_training"):
        self.log_dir = Path(log_dir)
        self.log_dir.mkdir(exist_ok=True)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.log_file = self.log_dir / f"{experiment_name}_{timestamp}.log"
        
        self.logger = logging.getLogger(experiment_name)
        self.logger.setLevel(logging.INFO)
        self.logger.handlers = []
        
        fh = logging.FileHandler(self.log_file)
        fh.setLevel(logging.INFO)
        ch = logging.StreamHandler()
        ch.setLevel(logging.INFO)
        
        formatter = logging.Formatter('%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
        fh.setFormatter(formatter)
        ch.setFormatter(formatter)
        
        self.logger.addHandler(fh)
        self.logger.addHandler(ch)
        
        self.start_time = None
        self.phase_start = None
        
        self.info(f"Logging to: {self.log_file}")
        self.info("="*80)
    
    def info(self, message):
        self.logger.info(message)
    
    def start_phase(self, phase_name):
        self.phase_start = time.time()
        self.info(f"\n{'='*80}")
        self.info(f"Starting: {phase_name}")
        self.info(f"{'='*80}")
    
    def end_phase(self, phase_name):
        if self.phase_start is not None:
            duration = time.time() - self.phase_start
            self.info(f"\nCompleted: {phase_name}")
            self.info(f"Duration: {self.format_duration(duration)}")
            self.info(f"{'='*80}")
            self.phase_start = None
    
    @staticmethod
    def format_duration(seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        if hours > 0:
            return f"{hours:02d}:{minutes:02d}:{secs:02d}"
        else:
            return f"{minutes:02d}:{secs:02d}"
    
    def start_training(self, num_epochs, phase_name="Training"):
        self.training_start = time.time()
        self.num_epochs = num_epochs
        self.start_phase(phase_name)
    
    def log_progress(self, epoch, num_epochs, train_loss, test_loss=None, train_r2=None, test_r2=None):
        elapsed = time.time() - self.training_start
        epochs_done = epoch + 1
        time_per_epoch = elapsed / epochs_done
        eta_seconds = time_per_epoch * (num_epochs - epochs_done)
        
        progress_pct = (epochs_done / num_epochs) * 100
        
        msg = f"Epoch {epoch}/{num_epochs} ({progress_pct:.1f}%) | "
        msg += f"Train Loss: {train_loss:.6f}"
        
        if test_loss is not None:
            msg += f" | Test Loss: {test_loss:.6f}"
        if train_r2 is not None:
            msg += f" | Train R²: {train_r2:.6f}"
        if test_r2 is not None:
            msg += f" | Test R²: {test_r2:.6f}"
        
        msg += f" | Elapsed: {self.format_duration(elapsed)}"
        msg += f" | ETA: {self.format_duration(eta_seconds)}"
        
        self.info(msg)
    
    def end_training(self, phase_name="Training"):
        total_time = time.time() - self.training_start
        self.info(f"\n{phase_name} completed in {self.format_duration(total_time)}")
        self.end_phase(phase_name)

# Initialize logger
logger = ExperimentLogger(log_dir="logs", experiment_name="multisession_training")
logger.info(f"Device: {DEVICE}")
logger.info(f"Window size: {WINDOW_SIZE}")

2025-10-29 15:45:06 - Logging to: logs/multisession_training_20251029_154506.log
2025-10-29 15:45:06 - ================================================================================
2025-10-29 15:45:06 - Device: cuda
2025-10-29 15:45:06 - Window size: 184


## Training Helper Function

In [ ]:
import builtins

original_train = multisession.train_from_cfg

def train_with_logging(*args, **kwargs):
    """Wrapper for train_from_cfg that logs progress to logger and saves best model to disk.

    This wrapper intercepts printed progress lines and keeps track of the best test R².
    When a new best model is found it saves a checkpoint under `saved_models/` and
    after training it also saves the final model state.
    """
    import os
    import time
    import torch

    original_print = builtins.print
    
    # Track best model
    best_test_r2 = -1e99
    best_model_state = None
    best_embeddings_stim = None
    test_r2_history = []
    
    # Extract model reference for checkpointing
    cfg = args[0] if len(args) > 0 else kwargs.get('cfg')
    model = args[1] if len(args) > 1 else kwargs.get('model')
    save_best = kwargs.pop('save_best_model', True)
    training_type = kwargs.pop('training_type', 'Unknown')  # NEW: training type for naming

    def _ensure_saved_models_dir():
        out_dir = os.path.join(os.getcwd(), 'saved_models')
        os.makedirs(out_dir, exist_ok=True)
        return out_dir

    def logging_print(*print_args, **print_kwargs):
        nonlocal best_test_r2, best_model_state, best_embeddings_stim
        msg = ' '.join(map(str, print_args))
        
        if msg.startswith('----'):
            parts = msg.split()
            if len(parts) >= 5:
                try:
                    epoch = int(parts[1])
                    train_loss = float(parts[2])
                    test_loss = float(parts[3])
                    train_r2 = float(parts[4])
                    test_r2 = float(parts[5]) if len(parts) > 5 else None
                    
                    logger.log_progress(epoch, kwargs.get('epochs', 7001), train_loss, test_loss, train_r2, test_r2)
                    
                    # Track best model and save to disk when improved
                    if save_best and test_r2 is not None:
                        test_r2_history.append((epoch, test_r2))
                        if test_r2 > best_test_r2:
                            best_test_r2 = test_r2
                            best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
                            logger.info(f"  → New best model! Test R²: {best_test_r2:.6f}")

                            # Save checkpoint to disk with training type in name
                            try:
                                out_dir = _ensure_saved_models_dir()
                                fname = os.path.join(out_dir, f"{training_type}_{model.__class__.__name__}_best_{int(time.time())}.pt")
                                torch.save({
                                    'model_state_dict': best_model_state,
                                    'epoch': epoch,
                                    'test_r2': best_test_r2,
                                    'timestamp': time.time(),
                                    'training_type': training_type,
                                }, fname)
                                logger.info(f"Saved best checkpoint to {fname}")
                            except Exception as e:
                                logger.warning(f"Failed to save best checkpoint: {e}")
                    
                    return
                except (ValueError, IndexError):
                    pass
        
        if msg.strip() and not msg.startswith('Building') and not msg.startswith('BOOM'):
            logger.info(msg)
        original_print(*print_args, **print_kwargs)
    
    builtins.print = logging_print
    
    try:
        embeddings_stim, results = original_train(*args, **kwargs)
        
        # Restore best model
        if save_best and best_model_state is not None:
            logger.info(f"Restoring best model (Test R²: {best_test_r2:.6f})")
            device = model.device
            model.load_state_dict({k: v.to(device) for k, v in best_model_state.items()})
            # Update results with best model info
            results['best_test_r2'] = best_test_r2
            results['final_test_r2'] = best_test_r2  # Override with best

        # Save final model to disk with training type in name
        try:
            out_dir = _ensure_saved_models_dir()
            final_fname = os.path.join(out_dir, f"{training_type}_{model.__class__.__name__}_final_{int(time.time())}.pt")
            model_cpu_state = {k: v.cpu() for k, v in model.state_dict().items()}
            torch.save({
                'model_state_dict': model_cpu_state,
                'results': results,
                'timestamp': time.time(),
                'training_type': training_type,
            }, final_fname)
            logger.info(f"Saved final model to {final_fname}")
        except Exception as e:
            logger.warning(f"Failed to save final model: {e}")

        # Clean up intermediate checkpoint files of the same training type only
        try:
            cleanup_checkpoints(training_type=training_type, model_name_pattern=model.__class__.__name__, 
                              keep_final=True, keep_latest_best=False)
        except Exception as e:
            logger.warning(f"Failed to cleanup checkpoints: {e}")

        return embeddings_stim, results
    finally:
        builtins.print = original_print

logger.info("Training helper function defined")

2025-10-29 15:45:06 - Training helper function defined


In [ ]:
import os
import glob
import time

def cleanup_checkpoints(training_type=None, model_name_pattern=None, keep_final=True, keep_latest_best=False):
    """
    Clean up checkpoint files after training completes.
    
    Args:
        training_type: Training type to match (e.g., "Baseline", "Spatial", "AE_Recon", "Sparsity")
                      If None, cleans all checkpoint files regardless of training type
        model_name_pattern: Pattern to match model names (e.g., "TBFMMultisession")
                           If None, cleans all checkpoint files
        keep_final: If True, keeps final model files (*_final_*.pt)
        keep_latest_best: If True, keeps the most recent best checkpoint
    """
    saved_models_dir = os.path.join(os.getcwd(), 'saved_models')
    if not os.path.exists(saved_models_dir):
        return
    
    # Build pattern to match files
    if training_type and model_name_pattern:
        pattern = os.path.join(saved_models_dir, f"{training_type}_{model_name_pattern}_*.pt")
    elif training_type:
        pattern = os.path.join(saved_models_dir, f"{training_type}_*.pt")
    elif model_name_pattern:
        pattern = os.path.join(saved_models_dir, f"*_{model_name_pattern}_*.pt")
    else:
        pattern = os.path.join(saved_models_dir, "*.pt")
    
    checkpoint_files = glob.glob(pattern)
    
    files_to_delete = []
    final_files = []
    best_files = []
    
    # Categorize files
    for filepath in checkpoint_files:
        filename = os.path.basename(filepath)
        if "_final_" in filename:
            final_files.append(filepath)
        elif "_best_" in filename or "_restored_best_" in filename:
            best_files.append(filepath)
        else:
            # Other checkpoint files
            files_to_delete.append(filepath)
    
    # Sort best files by timestamp (filename contains Unix timestamp)
    best_files.sort(key=lambda x: os.path.getmtime(x))
    
    # Determine what to delete
    if not keep_final:
        files_to_delete.extend(final_files)
    
    if not keep_latest_best:
        files_to_delete.extend(best_files)
    else:
        # Keep only the most recent best file
        files_to_delete.extend(best_files[:-1])
    
    # Delete files
    deleted_count = 0
    freed_mb = 0.0
    
    for filepath in files_to_delete:
        try:
            file_size = os.path.getsize(filepath) / (1024 * 1024)  # MB
            os.remove(filepath)
            filename = os.path.basename(filepath)
            logger.info(f"Deleted checkpoint: {filename}")
            deleted_count += 1
            freed_mb += file_size
        except Exception as e:
            logger.warning(f"Failed to delete {filepath}: {e}")
    
    if deleted_count > 0:
        type_msg = f"({training_type} only)" if training_type else "(all types)"
        logger.info(f"✅ Cleaned up {deleted_count} checkpoint files {type_msg} ({freed_mb:.1f} MB freed)")
    else:
        logger.info("No checkpoint files to clean up")

logger.info("Checkpoint cleanup function defined")

2025-10-29 15:45:06 - Checkpoint cleanup function defined


In [ ]:
# Test cleanup - clean up existing checkpoint files from TBFMMultisession training
# This will keep only the final models and remove intermediate checkpoints
cleanup_checkpoints(model_name_pattern="TBFMMultisession", keep_final=True, keep_latest_best=False)

2025-10-29 15:45:06 - Deleted checkpoint: Baseline_TBFMMultisession_best_1761777698.pt
2025-10-29 15:45:06 - Deleted checkpoint: Baseline_TBFMMultisession_best_1761777770.pt
2025-10-29 15:45:06 - ✅ Cleaned up 2 checkpoint files (all types) (0.0 MB freed)


## Checkpoint Management Solution ✅

**Problem**: Training functions were saving multiple checkpoint files (`_best_*.pt`) during training, but these intermediate files were not being cleaned up after training completed. This led to accumulated disk usage from old checkpoints.

**Solution**: 
1. **Added `cleanup_checkpoints()` function** that automatically removes intermediate checkpoint files
2. **Modified training functions** to call cleanup after training completes
3. **Keeps important files**: Only deletes intermediate `_best_*` checkpoints, preserves `_final_*` models

**What gets cleaned up:**
- ❌ `ModelName_best_timestamp.pt` (intermediate checkpoints)
- ❌ `ModelName_restored_best_timestamp.pt` (restored checkpoints) 
- ✅ `ModelName_final_timestamp.pt` (final models - **KEPT**)

**Modified functions:**
- `train_with_logging()` - baseline training wrapper
- `train_with_channel_sparsity()` - sparsity training function

**Result**: Training now automatically cleans up old checkpoints when each model finishes training, keeping only the final trained models.

In [ ]:
# Verify current checkpoint files - should only show final models now
print("Current model files in saved_models/:")
print("="*50)

import os
saved_models_dir = os.path.join(os.getcwd(), 'saved_models')
if os.path.exists(saved_models_dir):
    files = [f for f in os.listdir(saved_models_dir) if f.endswith('.pt')]
    files.sort()
    
    total_size_mb = 0
    for f in files:
        filepath = os.path.join(saved_models_dir, f)
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        total_size_mb += size_mb
        print(f"✅ {f:<50} ({size_mb:>7.1f} MB)")
    
    print("="*50)
    print(f"Total: {len(files)} files, {total_size_mb:.1f} MB")
    print("\n🎯 Only final models are kept - intermediate checkpoints removed!")
else:
    print("No saved_models directory found")

Current model files in saved_models/:
✅ Baseline_TBFMMultisession_final_1761736616.pt      ( 1683.6 MB)
✅ Baseline_TBFMMultisession_final_1761766970.pt      ( 1683.6 MB)
✅ Baseline_TBFMMultisession_final_1761777099.pt      (  855.7 MB)
✅ TBFMMultisession_best_1761722291.pt                (    0.0 MB)
✅ TBFMMultisession_final_1761688839.pt               ( 1683.6 MB)
✅ TBFMMultisession_final_1761689517.pt               (    0.1 MB)
✅ TBFMMultisession_final_1761711001.pt               ( 1683.6 MB)
✅ TBFMMultisession_final_1761712710.pt               ( 1683.6 MB)
✅ baseline_model_best.pt                             ( 1683.6 MB)
✅ baseline_model_best_5s.pt                          ( 3956.1 MB)
✅ compressed_sparsity_TBFMMultisession_final_1761735966.pt ( 1683.6 MB)
✅ compressed_sparsity_TBFMMultisession_final_1761768325.pt ( 1683.6 MB)
✅ compressed_sparsity_TBFMMultisession_final_1761776251.pt ( 1683.6 MB)
✅ identity_sparsity_TBFMMultisession_final_1761732723.pt ( 1683.6 MB)
✅ identity_spars

## Load Data

In [ ]:
logger.start_phase("Data Loading")

# held_in_session_ids = ["MonkeyG_20150925_Session2_S1", "MonkeyG_20150918_Session1_M1"]
held_in_session_ids=[
    "MonkeyG_20150914_Session1_S1",
    # "MonkeyG_20150915_Session3_S1",
    # "MonkeyG_20150915_Session5_S1",
    # "MonkeyG_20150916_Session4_S1",
    # "MonkeyG_20150917_Session1_M1",
    # "MonkeyG_20150917_Session1_S1",
    # "MonkeyG_20150917_Session2_M1",
    # "MonkeyG_20150917_Session2_S1",
    # "MonkeyG_20150921_Session3_S1",
    # "MonkeyG_20150921_Session5_S1",
    # "MonkeyG_20150922_Session1_S1",
    # "MonkeyG_20150922_Session2_S1",
    # "MonkeyG_20150925_Session1_S1",
    # "MonkeyG_20150925_Session2_S1",
    # "MonkeyJ_20160426_Session2_S1",
    # "MonkeyJ_20160426_Session3_S1",
    # "MonkeyJ_20160428_Session3_S1",
    # "MonkeyJ_20160429_Session1_S1",
    # "MonkeyJ_20160502_Session1_S1",
    # "MonkeyJ_20160624_Session3_S1",
    # "MonkeyJ_20160625_Session4_S1",
    # "MonkeyJ_20160625_Session5_S1",
    # "MonkeyJ_20160627_Session1_S1",
    # "MonkeyJ_20160630_Session3_S1",
    # "MonkeyJ_20160702_Session2_S1",
]


num_sessions = len(held_in_session_ids)
MAX_BATCH_SIZE = 62500 // 2
batch_size = (MAX_BATCH_SIZE // num_sessions) * num_sessions

d, held_out_session_ids = multisession.load_stim_batched(
    window_size=WINDOW_SIZE,
    session_subdir="torchraw",
    data_dir=DATA_DIR,
    unpack_stiminds=True,
    held_in_session_ids=held_in_session_ids,
    batch_size=batch_size,
    num_held_out_sessions=NUM_HELD_OUT_SESSIONS,
)
data_train, data_test = d.train_test_split(5000, test_cut=2500)

held_in_session_ids = data_train.session_ids
embeddings_rest = multisession.load_rest_embeddings(held_in_session_ids, device=DEVICE)

logger.info(f"Loaded {len(held_in_session_ids)} sessions")
logger.info(f"Sessions: {held_in_session_ids}")
logger.info(f"Training batch size: {batch_size}")

b = next(iter(data_train))
k0 = list(b.keys())[0]
logger.info(f"Train batch shape: {b[k0][0].shape}")

b = next(iter(data_test))
logger.info(f"Test batch shape: {b[k0][0].shape}")

logger.end_phase("Data Loading")

2025-10-29 15:45:06 - 
2025-10-29 15:45:06 - Starting: Data Loading
2025-10-29 15:45:06 - ================================================================================


2025-10-29 15:45:07 - Loaded 1 sessions
2025-10-29 15:45:07 - Sessions: ['MonkeyG_20150914_Session1_S1']
2025-10-29 15:45:07 - Training batch size: 31250
2025-10-29 15:45:07 - Train batch shape: torch.Size([5000, 20, 93])
2025-10-29 15:45:07 - Test batch shape: torch.Size([2500, 20, 93])
2025-10-29 15:45:07 - 
Completed: Data Loading
2025-10-29 15:45:07 - Duration: 00:00
2025-10-29 15:45:07 - ================================================================================


## Training Configuration

Set which models to train and their hyperparameters here. Models with `TRAIN_*=False` will be skipped.

In [ ]:
# ============================================================================
# TRAINING CONFIGURATION - Edit this cell to control which models to train
# ============================================================================

# -----------------------------------------------------------------------------
# Model Selection - Set to False to skip training that model
# -----------------------------------------------------------------------------
TRAIN_BASELINE = True       # Standard TBFM with PCA-initialized AE
TRAIN_SPATIAL = False       # Spatial regularization (was underperforming)
TRAIN_AE_RECON = False       # AE reconstruction penalty only
TRAIN_SPARSITY = True       # Identity AE + sparsity-driven approach

# -----------------------------------------------------------------------------
# Global Training Settings
# -----------------------------------------------------------------------------
GLOBAL_CONFIG = {
    'latent_dim': 50,                # AE latent dimension
    'epochs': 5000,                  # Training epochs (adjust for quick testing)
    'test_interval': 1000,           # Evaluation frequency
    'lambda_fro': 0.04,              # TBFM Frobenius regularization
    'coadapt': True,                 # Whether AE trains with TBFM
}

# -----------------------------------------------------------------------------
# Baseline Model Configuration
# -----------------------------------------------------------------------------
BASELINE_CONFIG = {
    'warm_start_is_identity': False,  # Use PCA initialization
    'coadapt': GLOBAL_CONFIG['coadapt'],
    'epochs': GLOBAL_CONFIG['epochs'],
}

# -----------------------------------------------------------------------------
# Spatial Regularization Configuration
# -----------------------------------------------------------------------------
SPATIAL_CONFIG = {
    'use_spatial': True,
    'lambda_spatial_smooth': 0.01,    # Spatial smoothness on reconstructions
    'lambda_spatial_decoder': 0.002,  # Spatial smoothness on decoder weights
    'spatial_penalty_freq': 10,       # Compute spatial penalties every N iters
    'coadapt': GLOBAL_CONFIG['coadapt'],
    'epochs': GLOBAL_CONFIG['epochs'],
}

# -----------------------------------------------------------------------------
# AE Reconstruction Configuration
# -----------------------------------------------------------------------------
AE_RECON_CONFIG = {
    'use_spatial': False,             # No spatial features
    'lambda_ae_recon': 0.05,          # AE reconstruction penalty weight
    'coadapt': GLOBAL_CONFIG['coadapt'],
    'epochs': GLOBAL_CONFIG['epochs'],
}

# -----------------------------------------------------------------------------
# Sparsity-Based Configuration
# -----------------------------------------------------------------------------
SPARSITY_CONFIG = {
    'warm_start_is_identity': True,   # Start with identity AE (no compression)
    'coadapt': False,                 # Freeze AE during Stage 1
    'lambda_channel_sparsity': 0.005, # L1 penalty on channel activations
    'lambda_sparsity_recon': 0.05,    # Sparsity-weighted reconstruction penalty
    'epochs': GLOBAL_CONFIG['epochs'],
}

# -----------------------------------------------------------------------------
# Print Configuration Summary
# -----------------------------------------------------------------------------
print("="*80)
print("TRAINING CONFIGURATION")
print("="*80)
print("\nModels to Train:")
print(f"  {'✓' if TRAIN_BASELINE else '✗'} Baseline (PCA AE)")
print(f"  {'✓' if TRAIN_SPATIAL else '✗'} Spatial Regularization")
print(f"  {'✓' if TRAIN_AE_RECON else '✗'} AE Reconstruction Penalty")
print(f"  {'✓' if TRAIN_SPARSITY else '✗'} Sparsity-Based (Identity AE)")

print(f"\nGlobal Settings:")
print(f"  Latent dim: {GLOBAL_CONFIG['latent_dim']}")
print(f"  Epochs: {GLOBAL_CONFIG['epochs']}")
print(f"  Test interval: {GLOBAL_CONFIG['test_interval']}")
print(f"  Coadapt: {GLOBAL_CONFIG['coadapt']}")
print("="*80)

TRAINING CONFIGURATION

Models to Train:
  ✓ Baseline (PCA AE)
  ✗ Spatial Regularization
  ✗ AE Reconstruction Penalty
  ✓ Sparsity-Based (Identity AE)

Global Settings:
  Latent dim: 50
  Epochs: 5000
  Test interval: 1000
  Coadapt: True


## Configure and Build Model

In [ ]:
if TRAIN_BASELINE:
    logger.start_phase("Model Configuration - Baseline")

    def cfg_base(cfg, dim):
        cfg.ae.training.coadapt = BASELINE_CONFIG['coadapt']
        cfg.ae.warm_start_is_identity = BASELINE_CONFIG['warm_start_is_identity']
        cfg.tbfm.module.use_film_bases = False
        cfg.tbfm.module.num_bases = 12
        cfg.tbfm.module.latent_dim = 64
        cfg.latent_dim = dim
        cfg.training.epochs = BASELINE_CONFIG['epochs']
        cfg.normalizers.module._target_ = "tbfm.normalizers.ScalerZscore"
        return cfg

    cfg = cfg_base(cfg, dim=GLOBAL_CONFIG['latent_dim'])
    cfg.tbfm.training.lambda_fro = GLOBAL_CONFIG['lambda_fro']

    logger.info(f"Configuration:")
    logger.info(f"  Epochs: {cfg.training.epochs}")
    logger.info(f"  Latent dim: {cfg.latent_dim}")
    logger.info(f"  Co-adaptation: {cfg.ae.training.coadapt}")
    logger.info(f"  Lambda Fro: {cfg.tbfm.training.lambda_fro}")

    logger.info("Building model...")
    ms = multisession.build_from_cfg(cfg, data_train, device=DEVICE)
    model_optims = multisession.get_optims(cfg, ms)

    logger.end_phase("Model Configuration - Baseline")
else:
    logger.info("⏭️  Skipping Baseline model (TRAIN_BASELINE=False)")

2025-10-29 15:45:07 - 
2025-10-29 15:45:07 - Starting: Model Configuration - Baseline
2025-10-29 15:45:07 - ================================================================================
2025-10-29 15:45:07 - Configuration:
2025-10-29 15:45:07 -   Epochs: 5000
2025-10-29 15:45:07 -   Latent dim: 50
2025-10-29 15:45:07 -   Co-adaptation: True
2025-10-29 15:45:07 -   Lambda Fro: 0.04
2025-10-29 15:45:07 - Building model...


Building and fitting normalizers...
Building and warm starting AEs...


2025-10-29 15:45:07 - 
Completed: Model Configuration - Baseline
2025-10-29 15:45:07 - Duration: 00:00
2025-10-29 15:45:07 - ================================================================================


Building TBFM...
BOOM! Dino DNA!


## Training

In [ ]:
if TRAIN_BASELINE:
    logger.start_training(cfg.training.epochs, "Baseline Model Training")

    embeddings_stim, results = train_with_logging(
        cfg,
        ms,
        data_train,
        model_optims,
        embeddings_rest,
        data_test=data_test,
        test_interval=GLOBAL_CONFIG['test_interval'],
        epochs=cfg.training.epochs,
        training_type="Baseline",  # NEW: Specify training type for model naming
        save_best_model=False
    )

    logger.end_training("Baseline Model Training")

    # Log final results
    logger.info("\n" + "="*80)
    logger.info("FINAL RESULTS - BASELINE")
    logger.info("="*80)
    logger.info(f"Overall Test R²: {results['final_test_r2']:.6f}")
    logger.info("\nPer-session Test R²:")
    for session_id, r2 in results['final_test_r2s'].items():
        logger.info(f"  {session_id}: {r2:.6f}")
    logger.info("="*80)
else:
    logger.info("⏭️  Skipping Baseline training (already skipped in config cell)")

2025-10-29 15:45:07 - 
2025-10-29 15:45:07 - Starting: Baseline Model Training
2025-10-29 15:45:07 - ================================================================================


2025-10-29 15:45:07 - Epoch 0/5000 (0.0%) | Train Loss: 1.222797 | Test Loss: 0.228147 | Train R²: 0.371025 | Test R²: 0.370803 | Elapsed: 00:00 | ETA: 08:20
2025-10-29 15:46:17 - Epoch 1000/5000 (20.0%) | Train Loss: 0.511797 | Test Loss: 0.171010 | Train R²: 0.526792 | Test R²: 0.527594 | Elapsed: 01:09 | ETA: 04:37
2025-10-29 15:47:33 - Epoch 2000/5000 (40.0%) | Train Loss: 0.446897 | Test Loss: 0.168466 | Train R²: 0.594049 | Test R²: 0.534565 | Elapsed: 02:25 | ETA: 03:38


KeyboardInterrupt: 

: 

## Spatial Regularization Comparison

Now we'll train a model with spatial regularization and compare it to the baseline.

In [ ]:
if TRAIN_SPATIAL:
    logger.start_phase("Loading Spatial Structures")

    # Import spatial utilities
    sys.path.append("/home/danmuir")
    from utils import get_preprocess_info

    # Load spatial structures for each session
    spatial_structures = {}
    for session_id in held_in_session_ids:
        logger.info(f"Loading spatial structure for {session_id}")
        (
            preprocess,
            ch_from_orig,
            ch_to_orig,
            node_position_orig,  # Dict[int, tuple]: node_id -> (x, y)
            node_id_orig,
            ch_from_after,
            ch_to_after,
        ) = get_preprocess_info(session_id, subdir=DATA_DIR)

        # Store for later use
        spatial_structures[session_id] = {
            'node_position': node_position_orig,
            'node_position_after': preprocess.node_position,
            'mask_indices': list(preprocess.node_position.keys()),
            'num_channels': len(preprocess.node_position)
        }

        logger.info(f"  {session_id}: {len(node_position_orig)} total electrodes, "
                    f"{len(preprocess.node_position)} valid electrodes")

    logger.end_phase("Loading Spatial Structures")
else:
    logger.info("⏭️  Skipping Spatial model (TRAIN_SPATIAL=False)")

2025-10-29 13:28:23 - ⏭️  Skipping Spatial model (TRAIN_SPATIAL=False)


In [ ]:
if TRAIN_SPATIAL:
    logger.start_phase("Building Model with Spatial Regularization")

    # Build a new model with spatial support enabled
    cfg_spatial = OmegaConf.create(OmegaConf.to_container(cfg, resolve=True))
    cfg_spatial.ae.module.use_spatial = SPATIAL_CONFIG['use_spatial']
    cfg_spatial.training.epochs = SPATIAL_CONFIG['epochs']
    cfg_spatial.ae.training.coadapt = SPATIAL_CONFIG['coadapt']

    logger.info("Building spatial model...")
    ms_spatial = multisession.build_from_cfg(cfg_spatial, data_train, device=DEVICE)

    # Register spatial structures for each session's autoencoder
    logger.info("Registering spatial structures...")
    for session_id in held_in_session_ids:
        ae_instance = ms_spatial.ae.instances[session_id]
        spatial_info = spatial_structures[session_id]
        
        ae_instance.register_spatial_structure(
            node_position=spatial_info['node_position'],
            mask_indices=spatial_info['mask_indices']
        )
        logger.info(f"  Registered spatial structure for {session_id}")

    model_optims_spatial = multisession.get_optims(cfg_spatial, ms_spatial)

    logger.end_phase("Building Model with Spatial Regularization")
else:
    logger.info("⏭️  Skipping Spatial model build (already skipped)")

2025-10-29 13:28:24 - ⏭️  Skipping Spatial model build (already skipped)


In [ ]:
logger.start_phase("Creating Spatial Training Function with AE Reconstruction Penalty")

# Create a modified training function that adds spatial regularization + AE reconstruction penalty
import torch.nn as nn
from tbfm.multisession import split_support_query_sessions, r2_score

import os
import time
import torch

def train_from_cfg_spatial_ae_recon(
    cfg,
    model,
    data_train,
    model_optims,
    embeddings_rest,
    data_test=None,
    inner_steps=None,
    epochs=10000,
    test_interval=None,
    support_size=300,
    embed_stim_lr=None,
    embed_stim_weight_decay=None,
    device="cuda",
    grad_clip=None,
    alternating_updates=True,
    bw_steps_per_bg_step=None,
    model_save_path=None,
    lambda_spatial_smooth=0.05,  # weight for spatial smoothness penalty
    lambda_spatial_decoder=0.01,  # weight for decoder spatial penalty
    lambda_ae_recon=0.01,  # NEW: weight for autoencoder reconstruction penalty L_2(dec(enc(x)), x)
    spatial_penalty_freq=10,  # compute spatial penalties every N iterations (saves ~50% time)
    save_best_model=True,  # save best model during training
    training_type="Custom",  # NEW: training type for model naming
):
    """
    Modified training with spatial regularization + AE reconstruction penalty.
    
    The AE reconstruction penalty penalizes lossy conversions through the encoder-decoder bottleneck:
        L_ae_recon = || dec(enc(x)) - x ||_2^2
    
    This encourages the autoencoder to preserve information through the latent bottleneck.
    
    Performance optimization: Set spatial_penalty_freq > 1 to compute spatial
    penalties less frequently (e.g., every 10 iterations instead of every iteration).
    This provides significant speedup (~50% faster) with minimal impact on final performance.
    """
    # Same setup as original
    test_interval = test_interval or cfg.training.test_interval
    embed_stim_lr = embed_stim_lr or cfg.film.training.optim.lr
    embed_stim_weight_decay = embed_stim_weight_decay or cfg.film.training.optim.weight_decay
    inner_steps = inner_steps or cfg.film.training.inner_steps
    use_film = cfg.tbfm.module.use_film_bases
    bw_steps_per_bg_step = bw_steps_per_bg_step or cfg.training.bw_steps_per_bg_step
    grad_clip = grad_clip or cfg.training.grad_clip or 10.0
    epochs = epochs or cfg.training.epochs
    support_size = support_size or cfg.film.training.support_size
    device = model.device

    def _ensure_saved_models_dir():
        out_dir = os.path.join(os.getcwd(), 'saved_models')
        os.makedirs(out_dir, exist_ok=True)
        return out_dir

    embeddings_stim = None
    iter_train = iter(data_train)

    train_losses = []
    train_r2s = []
    test_losses = []
    test_r2s = []
    spatial_smooth_losses = []
    spatial_decoder_losses = []
    ae_recon_losses = []  # NEW: track AE reconstruction losses
    best_test_r2 = -1e99
    best_model_state = None
    best_embeddings_stim = None

    for eidx in range(epochs):
        model.train()
        iter_train, _data_train = utils.iter_loader(iter_train, data_train, device=device)

        support, query = split_support_query_sessions(_data_train, support_size=support_size)

        with torch.no_grad():
            y_query = {sid: d[2] for sid, d in query.items()}
            y_query = model.norms(y_query)

        if use_film:
            model.eval()
            embeddings_stim = film.inner_update_stopgrad(
                model, support, embeddings_rest, inner_steps=inner_steps,
                lr=embed_stim_lr, weight_decay=embed_stim_weight_decay,
            )
            model.train()

        model_optims.zero_grad(set_to_none=True)

        yhat_query = model(query, embeddings_rest=embeddings_rest, embeddings_stim=embeddings_stim)

        losses = {}
        r2_trains = []
        spatial_smooth_total = 0.0
        spatial_decoder_total = 0.0

        # NEW: Only compute spatial penalties every N iterations for speedup
        compute_spatial = (eidx % spatial_penalty_freq) == 0

        for sid, y in y_query.items():
            _loss = nn.MSELoss()(yhat_query[sid], y)
            losses[sid] = _loss

            # NEW: Add spatial smoothness penalty on reconstructions (if enabled this iteration)
            if compute_spatial:
                ae_instance = model.ae.instances[sid]
                if hasattr(ae_instance, 'use_spatial') and ae_instance.use_spatial:
                    # Spatial smoothness on reconstructions
                    yhat_flat = yhat_query[sid].flatten(0, 1)  # (B*T, C)
                    smooth_penalty = ae_instance.spatial_smoothness_penalty(yhat_flat, reduction='mean')
                    spatial_smooth_total += smooth_penalty

                    # Spatial decoder penalty
                    mask = list(range(yhat_query[sid].shape[-1]))
                    decoder_penalty = ae_instance.spatial_decoder_penalty(mask)
                    spatial_decoder_total += decoder_penalty

            r2_train = r2_score(
                yhat_query[sid].permute(0, 2, 1).flatten(end_dim=1),
                y.permute(0, 2, 1).flatten(end_dim=1),
            )
            r2_trains.append(r2_train.item())

        # Compute reconstruction loss (MSE only)
        mse_only = sum(losses.values()) / len(y_query)

        # Build total loss
        loss = mse_only

        # Add TBFM regularization
        tbfm_regs = model.model.get_weighting_reg()
        loss = loss + cfg.tbfm.training.lambda_fro * sum(tbfm_regs.values()) / len(y_query)

        tbfm_regs_ortho = model.model.get_basis_rms_reg()
        loss = loss + cfg.tbfm.training.lambda_ortho * sum(tbfm_regs_ortho.values()) / len(y_query)

        # Add AE reconstruction penalty (if requested)
        ae_loss_total = 0.0
        if lambda_ae_recon > 0:
            for sid, y in y_query.items():
                ae_instance = model.ae.instances[sid]
                y_flat = y.flatten(0, 1)
                y_recon = ae_instance.reconstruct(y_flat, mask=list(range(y.shape[-1])))
                ae_loss = nn.MSELoss()(y_recon, y_flat)
                ae_loss_total += ae_loss
            ae_loss_total = ae_loss_total / len(y_query)
            loss = loss + lambda_ae_recon * ae_loss_total
        
        ae_recon_losses.append((eidx, ae_loss_total.item() if isinstance(ae_loss_total, torch.Tensor) else 0.0))

        # Add spatial regularization (if computed this iteration)
        if compute_spatial and spatial_smooth_total > 0:
            loss = loss + lambda_spatial_smooth * (spatial_smooth_total / len(y_query))
            loss = loss + lambda_spatial_decoder * (spatial_decoder_total / len(y_query))
            spatial_smooth_losses.append((eidx, spatial_smooth_total.item() / len(y_query)))
            spatial_decoder_losses.append((eidx, spatial_decoder_total.item() / len(y_query)))

        # Log losses
        train_losses.append((eidx, mse_only.item()))

        loss.backward()

        if grad_clip is not None:
            model_optims.clip_grad(value=grad_clip)

        if alternating_updates:
            update_basis_gen = (eidx % bw_steps_per_bg_step) == 0 or eidx < 200
            if update_basis_gen:
                model_optims.step()
            else:
                model_optims.step(skip=["bg", "film"])
        else:
            model_optims.step()

        # Testing
        if data_test is not None and (eidx % test_interval) == 0:
            train_r2s.append((eidx, sum(r2_trains) / len(y_query)))

            model_optims.zero_grad(set_to_none=True)
            model.eval()

            test_r2s_session = {}
            with torch.no_grad():
                iter_test = iter(data_test)
                iter_test, _data_test = utils.iter_loader(iter_test, data_test, device=device)

                y_test = {sid: d[2] for sid, d in _data_test.items()}
                y_test = model.norms(y_test)

                yhat_test = model(_data_test, embeddings_rest=embeddings_rest, embeddings_stim=embeddings_stim)

                test_losses_session = {}
                for sid, y in y_test.items():
                    _loss = nn.MSELoss()(yhat_test[sid], y)
                    test_losses_session[sid] = _loss

                    r2_test = r2_score(
                        yhat_test[sid].permute(0, 2, 1).flatten(end_dim=1),
                        y.permute(0, 2, 1).flatten(end_dim=1),
                    )
                    test_r2s_session[sid] = r2_test.item()

                test_loss = sum(test_losses_session.values()) / len(y_test)
                test_r2 = sum(test_r2s_session.values()) / len(y_test)

                test_losses.append((eidx, test_loss.item()))
                test_r2s.append((eidx, test_r2))

            logger.log_progress(eidx, epochs, train_losses[-1][1], test_loss.item(), train_r2s[-1][1], test_r2)
            
            if save_best_model and test_r2 > best_test_r2:
                best_test_r2 = test_r2
                best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
                if embeddings_stim is not None:
                    best_embeddings_stim = {k: v.cpu().clone() if torch.is_tensor(v) else v 
                                           for k, v in embeddings_stim.items()}
                else:
                    best_embeddings_stim = None
                logger.info(f"  → New best model! Test R²: {best_test_r2:.6f}")

                # Save checkpoint to disk with training type in name
                try:
                    out_dir = _ensure_saved_models_dir()
                    fname = os.path.join(out_dir, f"{training_type}_{model.__class__.__name__}_best_{int(time.time())}.pt")
                    torch.save({
                        'model_state_dict': best_model_state,
                        'epoch': eidx,
                        'test_r2': best_test_r2,
                        'timestamp': time.time(),
                        'training_type': training_type,
                    }, fname)
                    logger.info(f"Saved best checkpoint to {fname}")
                except Exception as e:
                    logger.warning(f"Failed to save best checkpoint: {e}")
            
            model.train()

    # Restore best model
    if save_best_model and best_model_state is not None:
        logger.info(f"Restoring best model (Test R²: {best_test_r2:.6f})")
        model.load_state_dict({k: v.to(device) for k, v in best_model_state.items()})
        if best_embeddings_stim is not None:
            embeddings_stim = {k: v.to(device) if torch.is_tensor(v) else v 
                              for k, v in best_embeddings_stim.items()}

        # Save restored best model to disk with training type in name
        try:
            out_dir = _ensure_saved_models_dir()
            best_fname = os.path.join(out_dir, f"{training_type}_{model.__class__.__name__}_restored_best_{int(time.time())}.pt")
            model_cpu_state = {k: v.cpu() for k, v in model.state_dict().items()}
            torch.save({
                'model_state_dict': model_cpu_state,
                'best_test_r2': best_test_r2,
                'timestamp': time.time(),
                'training_type': training_type,
            }, best_fname)
            logger.info(f"Saved restored best model to {best_fname}")
        except Exception as e:
            logger.warning(f"Failed to save restored best model: {e}")

        # Save final model with training type in name
        try:
            out_dir = _ensure_saved_models_dir()
            final_fname = os.path.join(out_dir, f"{training_type}_{model.__class__.__name__}_final_{int(time.time())}.pt")
            model_cpu_state = {k: v.cpu() for k, v in model.state_dict().items()}
            torch.save({
                'model_state_dict': model_cpu_state,
                'results': {
                    'train_losses': train_losses,
                    'test_losses': test_losses,
                    'train_r2s': train_r2s,
                    'test_r2s': test_r2s,
                    'spatial_smooth_losses': spatial_smooth_losses,
                    'spatial_decoder_losses': spatial_decoder_losses,
                    'ae_recon_losses': ae_recon_losses,
                    'final_test_r2': best_test_r2,
                    'final_test_r2s': test_r2s_session,
                    'best_test_r2': best_test_r2,
                },
                'timestamp': time.time(),
                'training_type': training_type,
            }, final_fname)
            logger.info(f"Saved final model to {final_fname}")
        except Exception as e:
            logger.warning(f"Failed to save final model: {e}")

        # Clean up intermediate checkpoint files of the same training type only
        try:
            cleanup_checkpoints(training_type=training_type, model_name_pattern=model.__class__.__name__, 
                              keep_final=True, keep_latest_best=False)
        except Exception as e:
            logger.warning(f"Failed to cleanup checkpoints: {e}")

    return embeddings_stim, {
        'train_losses': train_losses,
        'test_losses': test_losses,
        'train_r2s': train_r2s,
        'test_r2s': test_r2s,
        'spatial_smooth_losses': spatial_smooth_losses,
        'spatial_decoder_losses': spatial_decoder_losses,
        'ae_recon_losses': ae_recon_losses,
        'final_test_r2': best_test_r2,
        'final_test_r2s': test_r2s_session,
        'best_test_r2': best_test_r2,
    }

logger.end_phase("Creating Spatial Training Function with AE Reconstruction Penalty")

2025-10-29 13:28:24 - 
2025-10-29 13:28:24 - Starting: Creating Spatial Training Function with AE Reconstruction Penalty
2025-10-29 13:28:24 - ================================================================================
2025-10-29 13:28:24 - 
Completed: Creating Spatial Training Function with AE Reconstruction Penalty
2025-10-29 13:28:24 - Duration: 00:00
2025-10-29 13:28:24 - ================================================================================


In [ ]:
if TRAIN_SPATIAL:
    logger.start_training(cfg_spatial.training.epochs, "Spatial Model Training")

    # Train with spatial regularization
    embeddings_stim_spatial, results_spatial = train_from_cfg_spatial(
        cfg_spatial,
        ms_spatial,
        data_train,
        model_optims_spatial,
        embeddings_rest,
        data_test=data_test,
        test_interval=GLOBAL_CONFIG['test_interval'],
        epochs=cfg_spatial.training.epochs,
        lambda_spatial_smooth=SPATIAL_CONFIG['lambda_spatial_smooth'],
        lambda_spatial_decoder=SPATIAL_CONFIG['lambda_spatial_decoder'],
        spatial_penalty_freq=10,  # Compute spatial penalties every 10 iterations
    )

    logger.end_training("Spatial Model Training")

    # Log spatial results
    logger.info("\n" + "="*80)
    logger.info("SPATIAL MODEL RESULTS")
    logger.info("="*80)
    logger.info(f"Overall Test R²: {results_spatial['final_test_r2']:.6f}")
    logger.info("\nPer-session Test R²:")
    for session_id, r2 in results_spatial['final_test_r2s'].items():
        logger.info(f"  {session_id}: {r2:.6f}")
    logger.info("="*80)
else:
    logger.info("⏭️  Skipping Spatial training (already skipped)")

2025-10-29 13:28:24 - ⏭️  Skipping Spatial training (already skipped)


In [ ]:
if TRAIN_SPATIAL:
    # Save spatial model (best model is already loaded)
    spatial_model_path = model_save_dir / "spatial_model_best.pt"
    logger.info(f"Saving best spatial model to {spatial_model_path}")
    logger.info(f"Best Test R²: {results_spatial['best_test_r2']:.6f}")

    # Save model state
    torch.save({
        'ms_state_dict': ms_spatial.state_dict(),
        'embeddings_stim': embeddings_stim_spatial,
        'results': results_spatial,
        'config': OmegaConf.to_container(cfg_spatial, resolve=True),
        'held_in_session_ids': held_in_session_ids,
        'spatial_structures': spatial_structures,
        'best_test_r2': results_spatial['best_test_r2'],
    }, spatial_model_path)

    logger.info(f"Best spatial model saved successfully")
else:
    logger.info("⏭️  Skipping Spatial model save (already skipped)")

2025-10-29 13:28:24 - ⏭️  Skipping Spatial model save (already skipped)


## Train with AE Reconstruction Penalty (No Spatial)

Now let's train a model with the autoencoder reconstruction penalty added, without spatial regularization. This penalizes lossy conversions through the encoder-decoder bottleneck: L_ae_recon = || dec(enc(x)) - x ||_2^2

In [ ]:
if TRAIN_AE_RECON:
    logger.start_phase("Building Model with AE Reconstruction (No Spatial)")

    # Build a new model WITHOUT spatial support (like baseline)
    cfg_ae_recon = OmegaConf.create(OmegaConf.to_container(cfg, resolve=True))
    # Do NOT enable spatial features (keep use_spatial=False like baseline)
    cfg_ae_recon.training.epochs = AE_RECON_CONFIG['epochs']
    cfg_ae_recon.ae.training.coadapt = AE_RECON_CONFIG['coadapt']

    logger.info("Building AE reconstruction model (no spatial)...")
    ms_ae_recon = multisession.build_from_cfg(cfg_ae_recon, data_train, device=DEVICE)

    model_optims_ae_recon = multisession.get_optims(cfg_ae_recon, ms_ae_recon)

    logger.end_phase("Building Model with AE Reconstruction (No Spatial)")
else:
    logger.info("⏭️  Skipping AE Recon model (TRAIN_AE_RECON=False)")

2025-10-29 13:28:24 - ⏭️  Skipping AE Recon model (TRAIN_AE_RECON=False)


In [ ]:
if TRAIN_AE_RECON:
    logger.start_training(cfg_ae_recon.training.epochs, "AE Reconstruction Model Training (No Spatial)")

    # Train with ONLY AE reconstruction penalty (no spatial regularization)
    embeddings_stim_ae_recon, results_ae_recon = train_from_cfg_spatial_ae_recon(
        cfg_ae_recon,
        ms_ae_recon,
        data_train,
        model_optims_ae_recon,
        embeddings_rest,
        data_test=data_test,
        test_interval=GLOBAL_CONFIG['test_interval'],
        epochs=cfg_ae_recon.training.epochs,
        lambda_spatial_smooth=0.0,  # NO spatial smoothness
        lambda_spatial_decoder=0.0,  # NO spatial decoder penalty
        lambda_ae_recon=AE_RECON_CONFIG['lambda_ae_recon'],
        spatial_penalty_freq=10,
    )

    logger.end_training("AE Reconstruction Model Training (No Spatial)")

    # Log results
    logger.info("\n" + "="*80)
    logger.info("AE RECONSTRUCTION MODEL RESULTS (No Spatial)")
    logger.info("="*80)
    logger.info(f"Overall Test R²: {results_ae_recon['final_test_r2']:.6f}")
    logger.info("\nPer-session Test R²:")
    for session_id, r2 in results_ae_recon['final_test_r2s'].items():
        logger.info(f"  {session_id}: {r2:.6f}")
    logger.info("="*80)
else:
    logger.info("⏭️  Skipping AE Recon training (already skipped)")

2025-10-29 13:28:24 - ⏭️  Skipping AE Recon training (already skipped)


## Sparsity-Based Compression Approach

Since TBFM performs better with identity AE (no lossy compression), we'll use a two-stage approach:
1. **Stage 1**: Train with identity AE + L1 sparsity penalty on channels → learn which channels matter
2. **Stage 2**: Build selective AE that only compresses unimportant channels → task-informed compression

In [ ]:
logger.start_phase("Creating Sparsity Training Function")

# Create training function with L1 sparsity penalty on channel activations
import torch.nn as nn
from tbfm.multisession import split_support_query_sessions, r2_score

import os
import time
import torch

def train_with_channel_sparsity(
    cfg,
    model,
    data_train,
    model_optims,
    embeddings_rest,
    data_test=None,
    inner_steps=None,
    epochs=10000,
    test_interval=None,
    support_size=300,
    embed_stim_lr=None,
    embed_stim_weight_decay=None,
    device="cuda",
    grad_clip=None,
    alternating_updates=True,
    bw_steps_per_bg_step=None,
    lambda_channel_sparsity=0.001,  # L1 penalty on channel activations
    lambda_sparsity_recon=0.0,  # NEW: Sparsity-weighted AE reconstruction penalty
    save_best_model=True,
):
    """
    Train TBFM with L1 sparsity penalty on channel activations + sparsity-driven reconstruction.
    
    This encourages the model to:
    1. Use only a subset of channels (sparsity penalty)
    2. Preserve information in important channels (sparsity-weighted reconstruction)
    
    Args:
        lambda_channel_sparsity: Weight for L1 penalty on channel activations
                                Encourages many channels to have near-zero activation
        lambda_sparsity_recon: Weight for sparsity-weighted AE reconstruction penalty
                              Focuses reconstruction on high-activation (important) channels
    """
    # Same setup as original
    test_interval = test_interval or cfg.training.test_interval
    embed_stim_lr = embed_stim_lr or cfg.film.training.optim.lr
    embed_stim_weight_decay = embed_stim_weight_decay or cfg.film.training.optim.weight_decay
    inner_steps = inner_steps or cfg.film.training.inner_steps
    use_film = cfg.tbfm.module.use_film_bases
    bw_steps_per_bg_step = bw_steps_per_bg_step or cfg.training.bw_steps_per_bg_step
    grad_clip = grad_clip or cfg.training.grad_clip or 10.0
    epochs = epochs or cfg.training.epochs
    support_size = support_size or cfg.film.training.support_size
    device = model.device

    def _ensure_saved_models_dir():
        out_dir = os.path.join(os.getcwd(), 'saved_models')
        os.makedirs(out_dir, exist_ok=True)
        return out_dir

    embeddings_stim = None
    iter_train = iter(data_train)

    train_losses = []
    train_r2s = []
    test_losses = []
    test_r2s = []
    channel_sparsity_losses = []
    sparsity_recon_losses = []  # NEW: track sparsity-weighted reconstruction
    channel_activations = {sid: [] for sid in data_train.session_ids}
    best_test_r2 = -1e99
    best_model_state = None
    best_embeddings_stim = None

    for eidx in range(epochs):
        model.train()
        iter_train, _data_train = utils.iter_loader(iter_train, data_train, device=device)

        support, query = split_support_query_sessions(_data_train, support_size=support_size)

        with torch.no_grad():
            y_query = {sid: d[2] for sid, d in query.items()}
            y_query = model.norms(y_query)

        if use_film:
            model.eval()
            embeddings_stim = film.inner_update_stopgrad(
                model, support, embeddings_rest, inner_steps=inner_steps,
                lr=embed_stim_lr, weight_decay=embed_stim_weight_decay,
            )
            model.train()

        model_optims.zero_grad(set_to_none=True)

        yhat_query = model(query, embeddings_rest=embeddings_rest, embeddings_stim=embeddings_stim)

        losses = {}
        r2_trains = []
        sparsity_penalty_total = 0.0
        sparsity_recon_total = 0.0  # NEW

        for sid, y in y_query.items():
            _loss = nn.MSELoss()(yhat_query[sid], y)
            losses[sid] = _loss

            # Compute channel activations (importance)
            # yhat_query[sid] has shape (B, T, C) where C is channels
            channel_activation = yhat_query[sid].abs().mean(dim=(0, 1))  # (C,) - mean activation per channel
            
            # L1 sparsity penalty on channel activations
            sparsity_penalty = channel_activation.sum()
            sparsity_penalty_total += sparsity_penalty
            
            # NEW: Sparsity-weighted reconstruction penalty
            if lambda_sparsity_recon > 0:
                ae_instance = model.ae.instances[sid]
                mask = list(range(y.shape[-1]))  # All channels
                
                # Compute AE reconstruction: x_recon = dec(enc(x))
                y_flat = y.flatten(0, 1)  # (B*T, C)
                y_recon = ae_instance.reconstruct(y_flat, mask=mask)  # (B*T, C)
                
                # Compute per-channel reconstruction error
                channel_recon_error = (y_recon - y_flat).pow(2).mean(dim=0)  # (C,) - MSE per channel
                
                # Weight reconstruction error by channel importance
                # Important channels (high activation) → high penalty if reconstruction is poor
                # Unimportant channels (low activation) → low penalty even if reconstruction is poor
                # Normalize channel_activation to [0, 1] for stable weighting
                channel_weights = channel_activation / (channel_activation.max() + 1e-8)
                weighted_recon_error = (channel_weights * channel_recon_error).sum()
                
                sparsity_recon_total += weighted_recon_error
            
            # Track channel activations for analysis (every 100 epochs to save memory)
            if eidx % 100 == 0:
                channel_activations[sid].append((eidx, channel_activation.detach().cpu()))

            r2_train = r2_score(
                yhat_query[sid].permute(0, 2, 1).flatten(end_dim=1),
                y.permute(0, 2, 1).flatten(end_dim=1),
            )
            r2_trains.append(r2_train.item())

        # Compute reconstruction loss (MSE only)
        mse_only = sum(losses.values()) / len(y_query)

        # Build total loss
        loss = mse_only

        # Add TBFM regularization
        tbfm_regs = model.model.get_weighting_reg()
        loss = loss + cfg.tbfm.training.lambda_fro * sum(tbfm_regs.values()) / len(y_query)

        tbfm_regs_ortho = model.model.get_basis_rms_reg()
        loss = loss + cfg.tbfm.training.lambda_ortho * sum(tbfm_regs_ortho.values()) / len(y_query)

        # Add channel sparsity penalty
        if lambda_channel_sparsity > 0:
            loss = loss + lambda_channel_sparsity * (sparsity_penalty_total / len(y_query))

        # NEW: Add sparsity-weighted reconstruction penalty
        if lambda_sparsity_recon > 0:
            loss = loss + lambda_sparsity_recon * (sparsity_recon_total / len(y_query))

        # Log losses
        train_losses.append((eidx, mse_only.item()))
        channel_sparsity_losses.append((eidx, sparsity_penalty_total.item() / len(y_query)))
        sparsity_recon_losses.append((eidx, sparsity_recon_total.item() / len(y_query) if isinstance(sparsity_recon_total, torch.Tensor) else 0.0))

        loss.backward()

        if grad_clip is not None:
            model_optims.clip_grad(value=grad_clip)

        if alternating_updates:
            update_basis_gen = (eidx % bw_steps_per_bg_step) == 0 or eidx < 200
            if update_basis_gen:
                model_optims.step()
            else:
                model_optims.step(skip=["bg", "film"])
        else:
            model_optims.step()

        # Testing
        if data_test is not None and (eidx % test_interval) == 0:
            train_r2s.append((eidx, sum(r2_trains) / len(y_query)))

            model_optims.zero_grad(set_to_none=True)
            model.eval()

            test_r2s_session = {}
            with torch.no_grad():
                iter_test = iter(data_test)
                iter_test, _data_test = utils.iter_loader(iter_test, data_test, device=device)

                y_test = {sid: d[2] for sid, d in _data_test.items()}
                y_test = model.norms(y_test)

                yhat_test = model(_data_test, embeddings_rest=embeddings_rest, embeddings_stim=embeddings_stim)

                test_losses_session = {}
                for sid, y in y_test.items():
                    _loss = nn.MSELoss()(yhat_test[sid], y)
                    test_losses_session[sid] = _loss

                    r2_test = r2_score(
                        yhat_test[sid].permute(0, 2, 1).flatten(end_dim=1),
                        y.permute(0, 2, 1).flatten(end_dim=1),
                    )
                    test_r2s_session[sid] = r2_test.item()

                test_loss = sum(test_losses_session.values()) / len(y_test)
                test_r2 = sum(test_r2s_session.values()) / len(y_test)

                test_losses.append((eidx, test_loss.item()))
                test_r2s.append((eidx, test_r2))

            logger.log_progress(eidx, epochs, train_losses[-1][1], test_loss.item(), train_r2s[-1][1], test_r2)
            
            if save_best_model and test_r2 > best_test_r2:
                best_test_r2 = test_r2
                best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
                if embeddings_stim is not None:
                    best_embeddings_stim = {k: v.cpu().clone() if torch.is_tensor(v) else v 
                                           for k, v in embeddings_stim.items()}
                else:
                    best_embeddings_stim = None
                logger.info(f"  → New best model! Test R²: {best_test_r2:.6f}")

                # Save checkpoint to disk when improved
                try:
                    out_dir = _ensure_saved_models_dir()
                    fname = os.path.join(out_dir, f"{model.__class__.__name__}_best_{int(time.time())}.pt")
                    torch.save({
                        'model_state_dict': best_model_state,
                        'epoch': eidx,
                        'test_r2': best_test_r2,
                        'timestamp': time.time(),
                    }, fname)
                    logger.info(f"Saved best checkpoint to {fname}")
                except Exception as e:
                    logger.warning(f"Failed to save best checkpoint: {e}")
            
            model.train()

    # Restore best model
    if save_best_model and best_model_state is not None:
        logger.info(f"Restoring best model (Test R²: {best_test_r2:.6f})")
        model.load_state_dict({k: v.to(device) for k, v in best_model_state.items()})
        if best_embeddings_stim is not None:
            embeddings_stim = {k: v.to(device) if torch.is_tensor(v) else v 
                              for k, v in best_embeddings_stim.items()}

        # Save restored best model to disk
        try:
            out_dir = _ensure_saved_models_dir()
            best_fname = os.path.join(out_dir, f"{model.__class__.__name__}_restored_best_{int(time.time())}.pt")
            model_cpu_state = {k: v.cpu() for k, v in model.state_dict().items()}
            torch.save({
                'model_state_dict': model_cpu_state,
                'best_test_r2': best_test_r2,
                'timestamp': time.time(),
            }, best_fname)
            logger.info(f"Saved restored best model to {best_fname}")
        except Exception as e:
            logger.warning(f"Failed to save restored best model: {e}")

    # Also save final model file
    try:
        out_dir = _ensure_saved_models_dir()
        final_fname = os.path.join(out_dir, f"{model.__class__.__name__}_final_{int(time.time())}.pt")
        model_cpu_state = {k: v.cpu() for k, v in model.state_dict().items()}
        torch.save({
            'model_state_dict': model_cpu_state,
            'results': {
                'train_losses': train_losses,
                'test_losses': test_losses,
                'train_r2s': train_r2s,
                'test_r2s': test_r2s,
            },
            'timestamp': time.time(),
        }, final_fname)
        logger.info(f"Saved final model to {final_fname}")
    except Exception as e:
        logger.warning(f"Failed to save final model: {e}")

    # Clean up intermediate checkpoint files, keeping only the final model
    try:
        cleanup_checkpoints(model_name_pattern=model.__class__.__name__, 
                          keep_final=True, keep_latest_best=False)
    except Exception as e:
        logger.warning(f"Failed to cleanup checkpoints: {e}")

    return embeddings_stim, {
        'train_losses': train_losses,
        'test_losses': test_losses,
        'train_r2s': train_r2s,
        'test_r2s': test_r2s,
        'channel_sparsity_losses': channel_sparsity_losses,
        'sparsity_recon_losses': sparsity_recon_losses,
        'channel_activations': channel_activations,
        'final_test_r2': best_test_r2 if save_best_model else test_r2s[-1][1] if test_r2s else 0,
        'final_test_r2s': test_r2s_session if test_r2s else {},
        'best_test_r2': best_test_r2,
    }

2025-10-29 13:28:24 - 
2025-10-29 13:28:24 - Starting: Creating Sparsity Training Function
2025-10-29 13:28:24 - ================================================================================


### Stage 1: Train with Identity AE + Channel Sparsity

Use identity autoencoder (no compression) and apply L1 penalty to learn which channels are task-relevant.

In [ ]:
if TRAIN_SPARSITY:
    logger.start_phase("Building Model with Identity AE (Stage 1)")

    # Build model with identity AE (no coadaptation, stays identity)
    cfg_stage1 = OmegaConf.create(OmegaConf.to_container(cfg, resolve=True))
    cfg_stage1.ae.training.coadapt = SPARSITY_CONFIG['coadapt']
    cfg_stage1.ae.warm_start_is_identity = SPARSITY_CONFIG['warm_start_is_identity']
    cfg_stage1.training.epochs = SPARSITY_CONFIG['epochs']

    logger.info("Building Stage 1 model (identity AE)...")
    ms_stage1 = multisession.build_from_cfg(cfg_stage1, data_train, device=DEVICE)
    model_optims_stage1 = multisession.get_optims(cfg_stage1, ms_stage1)

    logger.info("Model configuration:")
    logger.info(f"  AE coadapt: {cfg_stage1.ae.training.coadapt}")
    logger.info(f"  AE identity init: {cfg_stage1.ae.warm_start_is_identity}")
    logger.info(f"  Epochs: {cfg_stage1.training.epochs}")

    logger.end_phase("Building Model with Identity AE (Stage 1)")
else:
    logger.info("⏭️  Skipping Sparsity model (TRAIN_SPARSITY=False)")

2025-10-29 13:28:24 - 
2025-10-29 13:28:24 - Starting: Building Model with Identity AE (Stage 1)
2025-10-29 13:28:24 - ================================================================================
2025-10-29 13:28:24 - Building Stage 1 model (identity AE)...


Building and fitting normalizers...


2025-10-29 13:28:24 - Model configuration:
2025-10-29 13:28:24 -   AE coadapt: False
2025-10-29 13:28:24 -   AE identity init: True
2025-10-29 13:28:24 -   Epochs: 5000
2025-10-29 13:28:24 - 
Completed: Building Model with Identity AE (Stage 1)
2025-10-29 13:28:24 - Duration: 00:00
2025-10-29 13:28:24 - ================================================================================


Building and warm starting AEs...
Building TBFM...
BOOM! Dino DNA!


In [ ]:
if TRAIN_SPARSITY:
    logger.info("⚠️  NOTE: This is the ORIGINAL (slower) sparsity approach")
    logger.info("⚠️  See 'Corrected Two-Stage Sparsity Approach' below for the faster method")
    logger.info("⚠️  Skipping original approach - run corrected version instead")
    
    # Uncomment below to run original approach for comparison
    # logger.start_training(cfg_stage1.training.epochs, "Stage 1: Identity AE + Sparsity Training")
    # 
    # # Train Stage 1 with channel sparsity penalty + sparsity-weighted reconstruction
    # embeddings_stim_s1, results_s1 = train_with_channel_sparsity(
    #     cfg_stage1,
    #     ms_stage1,
    #     data_train,
    #     model_optims_stage1,
    #     embeddings_rest,
    #     data_test=data_test,
    #     test_interval=GLOBAL_CONFIG['test_interval'],
    #     epochs=cfg_stage1.training.epochs,
    #     lambda_channel_sparsity=SPARSITY_CONFIG['lambda_channel_sparsity'],
    #     lambda_sparsity_recon=SPARSITY_CONFIG['lambda_sparsity_recon'],
    # )
    # 
    # logger.end_training("Stage 1: Identity AE + Sparsity Training")
    # 
    # # Log Stage 1 results
    # logger.info("\n" + "="*80)
    # logger.info("STAGE 1 RESULTS (Identity AE + Sparsity)")
    # logger.info("="*80)
    # logger.info(f"Overall Test R²: {results_s1['final_test_r2']:.6f}")
    # logger.info("\nPer-session Test R²:")
    # for session_id, r2 in results_s1['final_test_r2s'].items():
    #     logger.info(f"  {session_id}: {r2:.6f}")
    # logger.info("="*80)
else:
    logger.info("⏭️  Skipping Sparsity training (already skipped)")

2025-10-29 13:28:24 - ⚠️  NOTE: This is the ORIGINAL (slower) sparsity approach
2025-10-29 13:28:24 - ⚠️  See 'Corrected Two-Stage Sparsity Approach' below for the faster method
2025-10-29 13:28:24 - ⚠️  Skipping original approach - run corrected version instead


## Corrected Two-Stage Sparsity Approach

**Problem with Current Approach**: Training TBFM + AE simultaneously makes the AE reconstruction penalty expensive and the sparsity signals unclear.

**Correct Approach**:
1. **Stage 1**: Train TBFM bases with sparsity penalty (AE frozen as identity)
2. **Stage 2**: Train AE to compress the pretrained sparse TBFM bases

This separates concerns: first learn which channels/patterns matter, then learn efficient compression.

In [ ]:
# def train_tbfm_bases_only(
#     cfg,
#     model,
#     data_train,
#     model_optims,
#     embeddings_rest,
#     data_test=None,
#     epochs=5000,
#     test_interval=None,
#     lambda_channel_sparsity=0.005,
#     device="cuda",
#     save_best_model=True,
# ):
#     """
#     Stage 1: Train TBFM bases with sparsity penalty, keeping AE frozen as identity.
    
#     This focuses purely on learning which channels and temporal patterns matter
#     without the computational overhead of AE reconstruction penalties.
#     """
#     logger.info("Starting Stage 1: TBFM Basis Training (AE frozen)")
    
#     # Freeze AE parameters - keep as identity
#     for session_id in model.ae.instances:
#         ae_instance = model.ae.instances[session_id]
#         for param in ae_instance.parameters():
#             param.requires_grad = False
    
#     # Use existing model_optims but only update TBFM parameters
#     # (the frozen parameters just won't contribute gradients)
    
#     test_interval = test_interval or cfg.training.test_interval
#     use_film = cfg.tbfm.module.use_film_bases
#     support_size = cfg.film.training.support_size
    
#     train_losses = []
#     test_losses = []
#     train_r2s = []
#     test_r2s = []
#     sparsity_losses = []
#     best_test_r2 = -1e99
#     best_model_state = None
    
#     iter_train = iter(data_train)
    
#     for eidx in range(epochs):
#         model.train()
#         iter_train, _data_train = utils.iter_loader(iter_train, data_train, device=device)
        
#         support, query = split_support_query_sessions(_data_train, support_size=support_size)
        
#         with torch.no_grad():
#             y_query = {sid: d[2] for sid, d in query.items()}
#             y_query = model.norms(y_query)
        
#         # FiLM embeddings if needed
#         embeddings_stim = None
#         if use_film:
#             model.eval()
#             embeddings_stim = film.inner_update_stopgrad(
#                 model, support, embeddings_rest, 
#                 inner_steps=cfg.film.training.inner_steps,
#                 lr=cfg.film.training.optim.lr, 
#                 weight_decay=cfg.film.training.optim.weight_decay,
#             )
#             model.train()
        
#         model_optims.zero_grad(set_to_none=True)
        
#         # Forward pass through TBFM (AE is identity, so no compression)
#         yhat_query = model(query, embeddings_rest=embeddings_rest, embeddings_stim=embeddings_stim)
        
#         # Standard reconstruction loss
#         total_loss = 0
#         r2_trains = []
#         total_sparsity = 0
        
#         for sid, y in y_query.items():
#             mse_loss = nn.MSELoss()(yhat_query[sid], y)
#             total_loss += mse_loss
            
#             # Channel sparsity penalty - encourage sparse channel usage
#             channel_activation = yhat_query[sid].abs().mean()  # Overall activation sparsity
#             total_sparsity += channel_activation
            
#             # R2 for tracking
#             r2_train = r2_score(
#                 yhat_query[sid].permute(0, 2, 1).flatten(end_dim=1),
#                 y.permute(0, 2, 1).flatten(end_dim=1),
#             )
#             r2_trains.append(r2_train.item())
        
#         # Average losses
#         avg_loss = total_loss / len(y_query)
#         avg_sparsity = total_sparsity / len(y_query)
        
#         # Add TBFM regularization
#         tbfm_regs = model.model.get_weighting_reg()
#         reg_loss = cfg.tbfm.training.lambda_fro * sum(tbfm_regs.values()) / len(y_query)
        
#         tbfm_regs_ortho = model.model.get_basis_rms_reg()
#         ortho_loss = cfg.tbfm.training.lambda_ortho * sum(tbfm_regs_ortho.values()) / len(y_query)
        
#         # Total loss with sparsity
#         final_loss = avg_loss + reg_loss + ortho_loss + lambda_channel_sparsity * avg_sparsity
        
#         final_loss.backward()
        
#         # Use existing model_optims (frozen AE params won't get gradients)
#         model_optims.step()
        
#         # Logging
#         train_losses.append((eidx, avg_loss.item()))
#         sparsity_losses.append((eidx, avg_sparsity.item()))
        
#         # Testing
#         if data_test is not None and (eidx % test_interval) == 0:
#             train_r2s.append((eidx, sum(r2_trains) / len(r2_trains)))
            
#             model.eval()
#             with torch.no_grad():
#                 iter_test = iter(data_test)
#                 iter_test, _data_test = utils.iter_loader(iter_test, data_test, device=device)
                
#                 y_test = {sid: d[2] for sid, d in _data_test.items()}
#                 y_test = model.norms(y_test)
                
#                 yhat_test = model(_data_test, embeddings_rest=embeddings_rest, embeddings_stim=embeddings_stim)
                
#                 test_loss = 0
#                 test_r2s_session = {}
#                 for sid, y in y_test.items():
#                     loss = nn.MSELoss()(yhat_test[sid], y)
#                     test_loss += loss
                    
#                     r2_test = r2_score(
#                         yhat_test[sid].permute(0, 2, 1).flatten(end_dim=1),
#                         y.permute(0, 2, 1).flatten(end_dim=1),
#                     )
#                     test_r2s_session[sid] = r2_test.item()
                
#                 avg_test_loss = test_loss / len(y_test)
#                 avg_test_r2 = sum(test_r2s_session.values()) / len(test_r2s_session)
                
#                 test_losses.append((eidx, avg_test_loss.item()))
#                 test_r2s.append((eidx, avg_test_r2))
                
#                 logger.log_progress(eidx, epochs, avg_loss.item(), avg_test_loss.item(), 
#                                   train_r2s[-1][1], avg_test_r2)
                
#                 # Save best model
#                 if save_best_model and avg_test_r2 > best_test_r2:
#                     best_test_r2 = avg_test_r2
#                     best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
#                     logger.info(f"  → New best TBFM! Test R²: {best_test_r2:.6f}")
            
#             model.train()
    
#     # Restore best model
#     if save_best_model and best_model_state is not None:
#         logger.info(f"Restoring best TBFM model (Test R²: {best_test_r2:.6f})")
#         model.load_state_dict({k: v.to(device) for k, v in best_model_state.items()})
    
#     # Unfreeze AE for next stage
#     for session_id in model.ae.instances:
#         ae_instance = model.ae.instances[session_id]
#         for param in ae_instance.parameters():
#             param.requires_grad = True
    
#     logger.info("Stage 1 Complete: TBFM bases trained with sparsity")
    
#     return {
#         'train_losses': train_losses,
#         'test_losses': test_losses, 
#         'train_r2s': train_r2s,
#         'test_r2s': test_r2s,
#         'sparsity_losses': sparsity_losses,
#         'final_test_r2': best_test_r2 if save_best_model else test_r2s[-1][1] if test_r2s else 0,
#         'final_test_r2s': test_r2s_session if 'test_r2s_session' in locals() else {},
#     }

In [ ]:
# def train_ae_for_pretrained_tbfm(
#     cfg,
#     model,
#     data_train,
#     model_optims,
#     embeddings_rest,
#     data_test=None,
#     epochs=3000,
#     test_interval=None,
#     lambda_ae_recon=0.01,
#     device="cuda",
#     save_best_model=True,
# ):
#     """
#     Stage 2: Train AE to compress pretrained TBFM bases.
    
#     Now that we know which channels/patterns are important from Stage 1,
#     train the AE to efficiently compress the neural activity while preserving
#     the TBFM's learned representations.
#     """
#     logger.info("Starting Stage 2: AE Training (TBFM frozen)")
    
#     # Freeze TBFM parameters - keep learned bases fixed
#     for name, param in model.named_parameters():
#         if 'ae.' not in name:  # Freeze everything except AE
#             param.requires_grad = False
    
#     # Use existing model_optims but only AE parameters will get gradients
    
#     test_interval = test_interval or cfg.training.test_interval
#     use_film = cfg.tbfm.module.use_film_bases
#     support_size = cfg.film.training.support_size
    
#     train_losses = []
#     test_losses = []
#     train_r2s = []
#     test_r2s = []
#     ae_recon_losses = []
#     best_test_r2 = -1e99
#     best_model_state = None
    
#     iter_train = iter(data_train)
    
#     # Use the embeddings_stim from Stage 1 (don't update during Stage 2)
#     embeddings_stim = None
#     if use_film:
#         logger.info("Using frozen embeddings_stim from Stage 1 (no FiLM updates in Stage 2)")
#         # We'll compute embeddings_stim once and reuse it
#         model.eval()
#         with torch.no_grad():
#             iter_temp = iter(data_train)
#             iter_temp, _data_temp = utils.iter_loader(iter_temp, data_train, device=device)
#             support_temp, _ = split_support_query_sessions(_data_temp, support_size=support_size)
#             embeddings_stim = film.inner_update_stopgrad(
#                 model, support_temp, embeddings_rest, 
#                 inner_steps=cfg.film.training.inner_steps,
#                 lr=cfg.film.training.optim.lr, 
#                 weight_decay=cfg.film.training.optim.weight_decay,
#             )
#         model.train()
    
#     for eidx in range(epochs):
#         model.train()
#         iter_train, _data_train = utils.iter_loader(iter_train, data_train, device=device)
        
#         support, query = split_support_query_sessions(_data_train, support_size=support_size)
        
#         with torch.no_grad():
#             y_query = {sid: d[2] for sid, d in query.items()}
#             y_query = model.norms(y_query)
        
#         # Skip FiLM updates - use the frozen embeddings_stim computed above
        
#         model_optims.zero_grad(set_to_none=True)
        
#         # Forward pass - TBFM is frozen, only AE trains
#         yhat_query = model(query, embeddings_rest=embeddings_rest, embeddings_stim=embeddings_stim)
        
#         # Standard reconstruction loss
#         total_loss = 0
#         r2_trains = []
#         total_ae_recon = 0
        
#         for sid, y in y_query.items():
#             mse_loss = nn.MSELoss()(yhat_query[sid], y)
#             total_loss += mse_loss
            
#             # AE reconstruction penalty - ensure AE preserves information
#             if lambda_ae_recon > 0:
#                 ae_instance = model.ae.instances[sid]
#                 mask = list(range(y.shape[-1]))
                
#                 # Compute AE reconstruction error
#                 y_flat = y.flatten(0, 1)  # (B*T, C)
#                 y_recon = ae_instance.reconstruct(y_flat, mask=mask)
#                 ae_recon_loss = nn.MSELoss()(y_recon, y_flat)
#                 total_ae_recon += ae_recon_loss
            
#             # R2 for tracking
#             r2_train = r2_score(
#                 yhat_query[sid].permute(0, 2, 1).flatten(end_dim=1),
#                 y.permute(0, 2, 1).flatten(end_dim=1),
#             )
#             r2_trains.append(r2_train.item())
        
#         # Average losses
#         avg_loss = total_loss / len(y_query)
#         avg_ae_recon = total_ae_recon / len(y_query) if lambda_ae_recon > 0 else 0
        
#         # Total loss 
#         final_loss = avg_loss + lambda_ae_recon * avg_ae_recon
        
#         final_loss.backward()
        
#         # Use existing model_optims (frozen TBFM params won't get gradients)
#         model_optims.step()
        
#         # Logging
#         train_losses.append((eidx, avg_loss.item()))
#         ae_recon_losses.append((eidx, avg_ae_recon.item() if isinstance(avg_ae_recon, torch.Tensor) else avg_ae_recon))
        
#         # Testing
#         if data_test is not None and (eidx % test_interval) == 0:
#             train_r2s.append((eidx, sum(r2_trains) / len(r2_trains)))
            
#             model.eval()
#             with torch.no_grad():
#                 iter_test = iter(data_test)
#                 iter_test, _data_test = utils.iter_loader(iter_test, data_test, device=device)
                
#                 y_test = {sid: d[2] for sid, d in _data_test.items()}
#                 y_test = model.norms(y_test)
                
#                 yhat_test = model(_data_test, embeddings_rest=embeddings_rest, embeddings_stim=embeddings_stim)
                
#                 test_loss = 0
#                 test_r2s_session = {}
#                 for sid, y in y_test.items():
#                     loss = nn.MSELoss()(yhat_test[sid], y)
#                     test_loss += loss
                    
#                     r2_test = r2_score(
#                         yhat_test[sid].permute(0, 2, 1).flatten(end_dim=1),
#                         y.permute(0, 2, 1).flatten(end_dim=1),
#                     )
#                     test_r2s_session[sid] = r2_test.item()
                
#                 avg_test_loss = test_loss / len(y_test)
#                 avg_test_r2 = sum(test_r2s_session.values()) / len(test_r2s_session)
                
#                 test_losses.append((eidx, avg_test_loss.item()))
#                 test_r2s.append((eidx, avg_test_r2))
                
#                 logger.log_progress(eidx, epochs, avg_loss.item(), avg_test_loss.item(), 
#                                   train_r2s[-1][1], avg_test_r2)
                
#                 # Save best model
#                 if save_best_model and avg_test_r2 > best_test_r2:
#                     best_test_r2 = avg_test_r2
#                     best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
#                     logger.info(f"  → New best AE! Test R²: {best_test_r2:.6f}")
            
#             model.train()
    
#     # Restore best model
#     if save_best_model and best_model_state is not None:
#         logger.info(f"Restoring best AE model (Test R²: {best_test_r2:.6f})")
#         model.load_state_dict({k: v.to(device) for k, v in best_model_state.items()})
    
#     # Unfreeze all parameters for future use
#     for param in model.parameters():
#         param.requires_grad = True
    
#     logger.info("Stage 2 Complete: AE trained for pretrained TBFM")
    
#     return {
#         'train_losses': train_losses,
#         'test_losses': test_losses, 
#         'train_r2s': train_r2s,
#         'test_r2s': test_r2s,
#         'ae_recon_losses': ae_recon_losses,
#         'final_test_r2': best_test_r2 if save_best_model else test_r2s[-1][1] if test_r2s else 0,
#         'final_test_r2s': test_r2s_session if 'test_r2s_session' in locals() else {},
#     }

### Execute Corrected Two-Stage Training

Now let's run the proper sequence:
1. **Stage 1**: Learn sparse TBFM bases (much faster!)
2. **Stage 2**: Train AE to compress the learned bases

In [ ]:
if TRAIN_SPARSITY:
    # Import necessary modules for this stage
    from omegaconf import OmegaConf
    import torch
    from tbfm import multisession
    from tbfm.tbfm import TBFM
    from tbfm import utils
    from torch import nn
    import numpy as np
    
    logger.start_training(SPARSITY_CONFIG['epochs'], "Two-Stage Sparsity Training: Identity AE → Compressed AE")
    
    # STAGE 1: Train with Identity Autoencoder (equivalent to vanilla TBFM)
    logger.info("🚀 STAGE 1: Training with Identity Autoencoder (no compression)")
    logger.info("This is functionally equivalent to vanilla TBFM but uses existing infrastructure")
    
    # Create configuration for identity autoencoder
    cfg_identity = OmegaConf.create(OmegaConf.to_container(cfg, resolve=True))
    
    # Configure for identity transformation (no compression)  
    cfg_identity.ae.training.coadapt = SPARSITY_CONFIG['coadapt']
    cfg_identity.ae.warm_start_is_identity = True  # Start as identity
    
    # Get number of channels from existing batch sample
    # k0 is the first batch key, b[k0][0] gives us the data tensor
    num_channels = b[k0][0].shape[-1]  # Last dimension is channels (93)
    cfg_identity.ae.module.latent_dim = num_channels  # TRUE input dimension (93 channels)
    cfg_identity.training.epochs = SPARSITY_CONFIG['epochs']
    
    # Add strong sparsity penalties for Stage 1
    cfg_identity.training.lambda_sparse = SPARSITY_CONFIG.get('lambda_sparse', 0.02)
    cfg_identity.training.lambda_pred_sparse = SPARSITY_CONFIG.get('lambda_pred_sparse', 0.002)
    
    logger.info(f"  AE latent_dim: {cfg_identity.ae.module.latent_dim} (same as input - identity transform)")
    logger.info(f"  AE coadapt: {cfg_identity.ae.training.coadapt}")
    logger.info(f"  AE identity init: {cfg_identity.ae.warm_start_is_identity}")
    logger.info(f"  Sparsity lambda: {cfg_identity.training.lambda_sparse}")
    logger.info(f"  Prediction sparsity lambda: {cfg_identity.training.lambda_pred_sparse}")
    logger.info(f"  Epochs: {cfg_identity.training.epochs}")
    
    # Build identity model
    ms_identity = multisession.build_from_cfg(cfg_identity, data_train, device=DEVICE)
    model_optims_identity = multisession.get_optims(cfg_identity, ms_identity)
    
    # Train Stage 1: Identity AE with sparsity (equivalent to vanilla TBFM)
    logger.info("Training identity AE model with sparsity regularization...")
    embeddings_stim_identity, identity_results = train_with_logging(
        cfg_identity,
        ms_identity,
        data_train,
        model_optims_identity,
        embeddings_rest,
        data_test=data_test,
        epochs=cfg_identity.training.epochs,
        test_interval=GLOBAL_CONFIG['test_interval'],
        training_type="identity_sparsity",
        save_best_model=False
    )
    
    logger.info(f"✅ Stage 1 Complete - Identity AE (vanilla TBFM equiv) Test R²: {identity_results['final_test_r2']:.6f}")
    
    # STAGE 2: Train compressed model using the sparse TBFM learned in Stage 1
    logger.info("🚀 STAGE 2: Training compressed model with learned sparse TBFM knowledge")
    
    # Create configuration for compressed autoencoder
    cfg_compressed = OmegaConf.create(OmegaConf.to_container(cfg, resolve=True))
    cfg_compressed.ae.training.coadapt = True
    cfg_compressed.ae.warm_start_is_identity = False  # Now use actual compression
    cfg_compressed.ae.module.latent_dim = cfg.latent_dim  # Compressed dimension (e.g., 20)
    cfg_compressed.training.epochs = SPARSITY_CONFIG['epochs']  # Remaining epochs for Stage 2
    
    # Reduce sparsity penalties for Stage 2 (focus on compression)
    cfg_compressed.training.lambda_sparse = SPARSITY_CONFIG.get('lambda_sparse', 0.1) * 0.1
    cfg_compressed.training.lambda_pred_sparse = SPARSITY_CONFIG.get('lambda_pred_sparse', 0.01) * 0.1
    
    logger.info(f"  Compressed AE latent_dim: {cfg_compressed.ae.module.latent_dim}")
    logger.info(f"  Reduced sparsity lambda: {cfg_compressed.training.lambda_sparse}")
    logger.info(f"  Epochs: {cfg_compressed.training.epochs}")
    
    # Build compressed model
    ms_compressed = multisession.build_from_cfg(cfg_compressed, data_train, device=DEVICE)
    model_optims_compressed = multisession.get_optims(cfg_compressed, ms_compressed)
    
    # Transfer learned TBFM knowledge from identity model to compressed model
    logger.info("Transferring learned sparse TBFM knowledge to compressed model...")
    with torch.no_grad():
        for session_id in data_train.session_ids:
            identity_tbfm = ms_identity.model.instances[session_id]
            compressed_tbfm = ms_compressed.model.instances[session_id]
            
            # Copy TBFM basis generators (temporal patterns learned in Stage 1)
            if hasattr(identity_tbfm, 'bases') and hasattr(compressed_tbfm, 'bases'):
                compressed_tbfm.bases.load_state_dict(identity_tbfm.bases.state_dict())
                logger.info(f"  {session_id}: Transferred sparse TBFM basis generator")
            
            # Copy other TBFM components that are compatible
            if hasattr(identity_tbfm, 'norms') and hasattr(compressed_tbfm, 'norms'):
                compressed_tbfm.norms.load_state_dict(identity_tbfm.norms.state_dict())
    
    # Train Stage 2: Compressed model
    logger.info("Training compressed model with transferred sparse TBFM knowledge...")
    embeddings_stim_compressed, compressed_results = train_with_logging(
        cfg_compressed,
        ms_compressed,
        data_train,
        model_optims_compressed,
        embeddings_rest,
        data_test=data_test,
        epochs=cfg_compressed.training.epochs,
        test_interval=GLOBAL_CONFIG['test_interval'],
        training_type="compressed_sparsity",
    )
    
    logger.info(f"✅ Stage 2 Complete - Compressed Model Test R²: {compressed_results['final_test_r2']:.6f}")
    
    # Combine results for analysis
    results_corrected_sparsity = {
        'stage1_identity': identity_results,
        'stage2_compressed': compressed_results,
        'final_test_r2': compressed_results['final_test_r2'],
        'final_test_r2s': compressed_results.get('final_test_r2s', {}),
        'identity_model': ms_identity,
        'compressed_model': ms_compressed,
        'embeddings_stim_compressed': embeddings_stim_compressed,
    }
    
    logger.end_training("Two-Stage Sparsity Training")
    
    # Log final results
    logger.info("\n" + "="*80)
    logger.info("TWO-STAGE SPARSITY RESULTS")
    logger.info("="*80)
    logger.info(f"Stage 1 (Identity AE - Sparse TBFM): {identity_results['final_test_r2']:.6f}")
    logger.info(f"Stage 2 (Compressed AE): {compressed_results['final_test_r2']:.6f}")
    improvement = compressed_results['final_test_r2'] - identity_results['final_test_r2']
    logger.info(f"Compression Improvement: {improvement:+.6f}")
    
    if 'final_test_r2s' in compressed_results:
        logger.info("\nPer-session Final R² (Compressed):")
        for session_id, r2 in compressed_results['final_test_r2s'].items():
            identity_r2 = identity_results.get('final_test_r2s', {}).get(session_id, 0.0)
            improvement_session = r2 - identity_r2
            logger.info(f"  {session_id}: {r2:.6f} ({improvement_session:+.6f})")
    logger.info("="*80)
    
else:
    logger.info("⏭️  Skipping Sparsity training (TRAIN_SPARSITY=False)")

2025-10-29 13:28:24 - 
2025-10-29 13:28:24 - Starting: Two-Stage Sparsity Training: Identity AE → Compressed AE
2025-10-29 13:28:24 - ================================================================================
2025-10-29 13:28:24 - 🚀 STAGE 1: Training with Identity Autoencoder (no compression)
2025-10-29 13:28:24 - This is functionally equivalent to vanilla TBFM but uses existing infrastructure
2025-10-29 13:28:24 -   AE latent_dim: 93 (same as input - identity transform)
2025-10-29 13:28:24 -   AE coadapt: False
2025-10-29 13:28:24 -   AE identity init: True
2025-10-29 13:28:24 -   Sparsity lambda: 0.02
2025-10-29 13:28:24 -   Prediction sparsity lambda: 0.002
2025-10-29 13:28:24 -   Epochs: 5000


Building and fitting normalizers...
Building and warm starting AEs...


2025-10-29 13:28:25 - Training identity AE model with sparsity regularization...


Building TBFM...
BOOM! Dino DNA!


2025-10-29 13:28:26 - Epoch 0/5000 (0.0%) | Train Loss: 339.262268 | Test Loss: 12.612190 | Train R²: 0.037596 | Test R²: 0.061099 | Elapsed: 00:01 | ETA: 02:43:50
2025-10-29 13:28:26 -   → New best model! Test R²: 0.061099
2025-10-29 13:28:26 - Saved best checkpoint to /home/danmuir/GitHub/py-tbfm/saved_models/identity_sparsity_TBFMMultisession_best_1761769706.pt
2025-10-29 13:39:18 - Epoch 1000/5000 (20.0%) | Train Loss: 338.349976 | Test Loss: 12.383114 | Train R²: 0.039331 | Test R²: 0.088338 | Elapsed: 10:53 | ETA: 43:30
2025-10-29 13:39:18 -   → New best model! Test R²: 0.088338
2025-10-29 13:39:18 - Saved best checkpoint to /home/danmuir/GitHub/py-tbfm/saved_models/identity_sparsity_TBFMMultisession_best_1761770358.pt
2025-10-29 13:50:11 - Epoch 2000/5000 (40.0%) | Train Loss: 338.311554 | Test Loss: 12.290613 | Train R²: 0.039793 | Test R²: 0.100564 | Elapsed: 21:46 | ETA: 32:38
2025-10-29 13:50:11 -   → New best model! Test R²: 0.100564
2025-10-29 13:50:11 - Saved best checkpo

...

2025-10-29 14:23:13 - Transferring learned sparse TBFM knowledge to compressed model...
2025-10-29 14:23:13 -   MonkeyG_20150914_Session1_S1: Transferred sparse TBFM basis generator
2025-10-29 14:23:13 -   MonkeyG_20150915_Session3_S1: Transferred sparse TBFM basis generator
2025-10-29 14:23:13 - Training compressed model with transferred sparse TBFM knowledge...


Building TBFM...
BOOM! Dino DNA!


2025-10-29 14:23:14 - Epoch 0/5000 (0.0%) | Train Loss: 114.637749 | Test Loss: 24.307793 | Train R²: 0.355693 | Test R²: -1.298563 | Elapsed: 54:49 | ETA: 4567:23:48
2025-10-29 14:23:14 -   → New best model! Test R²: -1.298563
2025-10-29 14:23:14 - Saved best checkpoint to /home/danmuir/GitHub/py-tbfm/saved_models/compressed_sparsity_TBFMMultisession_best_1761772994.pt
2025-10-29 14:33:57 - Epoch 1000/5000 (20.0%) | Train Loss: 88.840675 | Test Loss: 5.496982 | Train R²: 0.593587 | Test R²: 0.483361 | Elapsed: 01:05:32 | ETA: 04:21:49
2025-10-29 14:33:57 -   → New best model! Test R²: 0.483361
2025-10-29 14:33:57 - Saved best checkpoint to /home/danmuir/GitHub/py-tbfm/saved_models/compressed_sparsity_TBFMMultisession_best_1761773637.pt
2025-10-29 14:44:31 - Epoch 2000/5000 (40.0%) | Train Loss: 87.502777 | Test Loss: 5.444959 | Train R²: 0.609203 | Test R²: 0.488699 | Elapsed: 01:16:06 | ETA: 01:54:03
2025-10-29 14:44:31 -   → New best model! Test R²: 0.488699
2025-10-29 14:44:31 - Sa

Final: 5.457650184631348 0.4941916763782501


2025-10-29 15:17:33 - Saved final model to /home/danmuir/GitHub/py-tbfm/saved_models/compressed_sparsity_TBFMMultisession_final_1761776251.pt
2025-10-29 15:17:33 - Deleted checkpoint: compressed_sparsity_TBFMMultisession_best_1761772994.pt
2025-10-29 15:17:33 - Deleted checkpoint: compressed_sparsity_TBFMMultisession_best_1761773637.pt
2025-10-29 15:17:33 - Deleted checkpoint: compressed_sparsity_TBFMMultisession_best_1761774271.pt
2025-10-29 15:17:33 - Deleted checkpoint: compressed_sparsity_TBFMMultisession_best_1761774922.pt
2025-10-29 15:17:33 - Deleted checkpoint: compressed_sparsity_TBFMMultisession_best_1761775588.pt
2025-10-29 15:17:33 - ✅ Cleaned up 5 checkpoint files (compressed_sparsity only) (0.0 MB freed)
2025-10-29 15:17:33 - ✅ Stage 2 Complete - Compressed Model Test R²: 0.498682
2025-10-29 15:17:33 - 
Two-Stage Sparsity Training completed in 01:49:08
2025-10-29 15:17:33 - 
Completed: Two-Stage Sparsity Training
2025-10-29 15:17:33 - Duration: 01:49:08
2025-10-29 15:17:3

### Why This Corrected Approach is Much Faster

**Original (Slow) Approach:**
```python
# EVERY epoch, EVERY session:
yhat = model(data)  # TBFM forward pass
y_recon = ae.reconstruct(y)  # ADDITIONAL AE forward pass  
channel_weights = yhat.abs().mean()  # Channel analysis
weighted_loss = channel_weights * reconstruction_error  # Complex weighting
```

**Corrected (Fast) Approach:**
```python
# Stage 1: Only TBFM training (no AE overhead)
yhat = model(data)  # TBFM forward pass only
sparsity_penalty = yhat.abs().mean()  # Simple sparsity

# Stage 2: Only AE training (TBFM frozen)
yhat = model(data)  # Single forward pass, TBFM cached
ae_loss = ae.reconstruct(y)  # AE only trains here
```

**Performance Benefits:**
- ⚡ **~50% faster**: No double forward passes during Stage 1
- 🎯 **Clearer objectives**: Each stage has one focused goal  
- 🧠 **Better learning**: TBFM learns patterns first, then AE compresses
- 💾 **Less memory**: No need to track channel activations during TBFM training

In [ ]:
# # Save baseline model (best model is already loaded)
# import os
# from pathlib import Path

# model_save_dir = Path("saved_models")
# model_save_dir.mkdir(exist_ok=True)

# baseline_model_path = model_save_dir / "baseline_model_best.pt"
# logger.info(f"Saving best baseline model to {baseline_model_path}")
# if 'best_test_r2' in results:
#     logger.info(f"Best Test R²: {results['best_test_r2']:.6f}")

# # Save model state
# torch.save({
#     'ms_state_dict': ms.state_dict(),
#     'embeddings_stim': embeddings_stim,
#     'results': results,
#     'config': OmegaConf.to_container(cfg, resolve=True),
#     'held_in_session_ids': held_in_session_ids,
#     'best_test_r2': results.get('best_test_r2', results.get('final_test_r2')),
# }, baseline_model_path)

# logger.info(f"Best baseline model saved successfully")

## Visualizations

In [ ]:
# Baseline training curves (original)
txt = [t[0] for t in results['train_losses']]
tlt = [t[1] for t in results['train_losses']]
plt.figure(figsize=(10, 5))
plt.plot(txt[-500:], tlt[-500:], label="train (last 500)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Baseline: Recent Training Loss")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Full baseline curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

tx = [t[0] for t in results['test_losses']]
tl = [t[1] for t in results['test_losses']]

axes[0].plot(txt, tlt, label="Baseline Train", alpha=0.7)
axes[0].plot(tx, tl, label="Baseline Test", alpha=0.7)
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("Baseline: Loss Curves")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

tx_r2 = [t[0] for t in results['train_r2s']]
tr = [t[1] for t in results['train_r2s']]
te = [t[1] for t in results['test_r2s']]

axes[1].plot(tx_r2, tr, label="Baseline Train", alpha=0.7)
axes[1].plot(tx_r2, te, label="Baseline Test", alpha=0.7)
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("R²")
axes[1].set_title("Baseline: R² Curves")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

NameError: name 'results' is not defined

: 

In [ ]:
# Visualize AE reconstruction penalty alongside other metrics
# Handle missing data gracefully
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Check which models are available
models_available = {
    'baseline': 'results' in globals(),
    'spatial': 'results_spatial' in globals(),
    'ae_recon': 'results_ae_recon' in globals(),
}

print("Available models:")
for name, available in models_available.items():
    print(f"  {name}: {'✓' if available else '✗'}")
print()

# Extract baseline data if available
if models_available['baseline']:
    txt = [t[0] for t in results['train_losses']]
    tlt = [t[1] for t in results['train_losses']]
    tx = [t[0] for t in results['test_losses']]
    tl = [t[1] for t in results['test_losses']]
    tx_r2 = [t[0] for t in results['train_r2s']]
    tr = [t[1] for t in results['train_r2s']]
    te = [t[1] for t in results['test_r2s']]

# Extract spatial data if available
if models_available['spatial']:
    txt_s = [t[0] for t in results_spatial['train_losses']]
    tlt_s = [t[1] for t in results_spatial['train_losses']]
    tx_s = [t[0] for t in results_spatial['test_losses']]
    tl_s = [t[1] for t in results_spatial['test_losses']]
    tx_r2_s = [t[0] for t in results_spatial['train_r2s']]
    tr_s = [t[1] for t in results_spatial['train_r2s']]
    te_s = [t[1] for t in results_spatial['test_r2s']]

# Extract AE recon data if available
if models_available['ae_recon']:
    txt_ae = [t[0] for t in results_ae_recon['train_losses']]
    tlt_ae = [t[1] for t in results_ae_recon['train_losses']]
    tx_ae = [t[0] for t in results_ae_recon['test_losses']]
    tl_ae = [t[1] for t in results_ae_recon['test_losses']]
    tx_r2_ae = [t[0] for t in results_ae_recon['train_r2s']]
    tr_ae = [t[1] for t in results_ae_recon['train_r2s']]
    te_ae = [t[1] for t in results_ae_recon['test_r2s']]

# Test Loss Comparison
if models_available['baseline']:
    axes[0, 0].plot(tx, tl, label="Baseline", alpha=0.8, linewidth=2, color='C0')
if models_available['spatial']:
    axes[0, 0].plot(tx_s, tl_s, label="Spatial", alpha=0.8, linewidth=2, color='C2')
if models_available['ae_recon']:
    axes[0, 0].plot(tx_ae, tl_ae, label="AE Recon", alpha=0.8, linewidth=2, color='C3')
axes[0, 0].set_xlabel("Epoch")
axes[0, 0].set_ylabel("Test Loss")
axes[0, 0].set_title("Test Loss: All Available Models")
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Test R² Comparison
if models_available['baseline']:
    axes[0, 1].plot(tx_r2, te, label="Baseline", alpha=0.8, linewidth=2, color='C0')
if models_available['spatial']:
    axes[0, 1].plot(tx_r2_s, te_s, label="Spatial", alpha=0.8, linewidth=2, color='C2')
if models_available['ae_recon']:
    axes[0, 1].plot(tx_r2_ae, te_ae, label="AE Recon", alpha=0.8, linewidth=2, color='C3')
axes[0, 1].set_xlabel("Epoch")
axes[0, 1].set_ylabel("Test R²")
axes[0, 1].set_title("Test R²: All Available Models")
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# AE Reconstruction penalty over training (if available)
if models_available['ae_recon']:
    ae_recon_x = [t[0] for t in results_ae_recon['ae_recon_losses']]
    ae_recon_y = [t[1] for t in results_ae_recon['ae_recon_losses']]
    axes[1, 0].plot(ae_recon_x, ae_recon_y, alpha=0.8, linewidth=2, color='C1')
    axes[1, 0].set_xlabel("Epoch")
    axes[1, 0].set_ylabel("AE Reconstruction Loss")
    axes[1, 0].set_title("AE Reconstruction Penalty Over Training")
    axes[1, 0].grid(True, alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, "AE Recon model not trained", 
                    ha='center', va='center', transform=axes[1, 0].transAxes)
    axes[1, 0].set_title("AE Reconstruction Penalty (N/A)")

# Per-session R² comparison bar chart
if any(models_available.values()):
    # Get session list from first available model
    if models_available['baseline']:
        sessions = list(results['final_test_r2s'].keys())
    elif models_available['spatial']:
        sessions = list(results_spatial['final_test_r2s'].keys())
    else:
        sessions = list(results_ae_recon['final_test_r2s'].keys())
    
    x = range(len(sessions))
    width = 0.25
    offset = 0
    
    if models_available['baseline']:
        baseline_vals = [results['final_test_r2s'][s] for s in sessions]
        axes[1, 1].bar([i + offset*width for i in x], baseline_vals, width, 
                      label='Baseline', color='C0', alpha=0.7)
        offset += 1
    
    if models_available['spatial']:
        spatial_vals = [results_spatial['final_test_r2s'][s] for s in sessions]
        axes[1, 1].bar([i + offset*width for i in x], spatial_vals, width, 
                      label='Spatial', color='C2', alpha=0.7)
        offset += 1
    
    if models_available['ae_recon']:
        ae_recon_vals = [results_ae_recon['final_test_r2s'][s] for s in sessions]
        axes[1, 1].bar([i + offset*width for i in x], ae_recon_vals, width, 
                      label='AE Recon', color='C3', alpha=0.7)
        offset += 1
    
    axes[1, 1].set_ylabel("Test R²")
    axes[1, 1].set_title("Per-Session Test R² Comparison")
    axes[1, 1].set_xticks([i + (offset-1)*width/2 for i in x])
    axes[1, 1].set_xticklabels([s.split('_')[1] + '_' + s.split('_')[-1] for s in sessions], 
                               rotation=45, ha='right')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3, axis='y')
else:
    axes[1, 1].text(0.5, 0.5, "No models trained yet", 
                    ha='center', va='center', transform=axes[1, 1].transAxes)
    axes[1, 1].set_title("Per-Session Comparison (N/A)")

plt.tight_layout()
plt.show()

# Print comparison table
print("\n" + "="*80)
print("FINAL COMPARISON: Available Models")
print("="*80)

comparison_data = []
if models_available['baseline']:
    comparison_data.append(("Baseline", results['final_test_r2']))
if models_available['spatial']:
    comparison_data.append(("Spatial", results_spatial['final_test_r2']))
if models_available['ae_recon']:
    comparison_data.append(("AE Recon", results_ae_recon['final_test_r2']))

if comparison_data:
    for name, r2 in comparison_data:
        print(f"{name:20s} Test R² = {r2:.6f}")
    
    if len(comparison_data) > 1:
        print("="*80)
        print("\nImprovement over Baseline:")
        baseline_r2 = comparison_data[0][1] if comparison_data[0][0] == "Baseline" else None
        if baseline_r2:
            for name, r2 in comparison_data[1:]:
                improvement = r2 - baseline_r2
                pct = improvement / baseline_r2 * 100
                print(f"  {name:18s} {improvement:+.6f} ({pct:+.2f}%)")
else:
    print("No models trained yet. Run training cells above.")

print("="*80)

In [ ]:
# Visualize training curves with both penalties
# Handle missing Stage 1 data

if 'results_s1' not in globals():
    print("="*80)
    print("Stage 1 not trained yet - run Stage 1 training cell first")
    print("="*80)
else:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # Test R² over training
    tx_r2_s1 = [t[0] for t in results_s1['train_r2s']]
    te_s1 = [t[1] for t in results_s1['test_r2s']]

    axes[0, 0].plot(tx_r2_s1, te_s1, label="Stage 1 (Sparsity)", linewidth=2, color='C3')
    axes[0, 0].set_xlabel("Epoch")
    axes[0, 0].set_ylabel("Test R²")
    axes[0, 0].set_title("Test R²: Sparsity-Based Training")
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Test Loss over training
    tx_s1 = [t[0] for t in results_s1['test_losses']]
    tl_s1 = [t[1] for t in results_s1['test_losses']]

    axes[0, 1].plot(tx_s1, tl_s1, label="Stage 1 (Sparsity)", linewidth=2, color='C3')
    axes[0, 1].set_xlabel("Epoch")
    axes[0, 1].set_ylabel("Test Loss")
    axes[0, 1].set_title("Test Loss: Sparsity-Based Training")
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Channel sparsity penalty over training
    sparsity_x = [t[0] for t in results_s1['channel_sparsity_losses']]
    sparsity_y = [t[1] for t in results_s1['channel_sparsity_losses']]

    axes[1, 0].plot(sparsity_x, sparsity_y, linewidth=2, color='C4')
    axes[1, 0].set_xlabel("Epoch")
    axes[1, 0].set_ylabel("Channel Sparsity Penalty (L1)")
    axes[1, 0].set_title("L1 Sparsity Penalty Over Training")
    axes[1, 0].grid(True, alpha=0.3)

    # Sparsity-weighted reconstruction penalty over training
    recon_x = [t[0] for t in results_s1['sparsity_recon_losses']]
    recon_y = [t[1] for t in results_s1['sparsity_recon_losses']]

    axes[1, 1].plot(recon_x, recon_y, linewidth=2, color='C5')
    axes[1, 1].set_xlabel("Epoch")
    axes[1, 1].set_ylabel("Sparsity-Weighted Reconstruction Loss")
    axes[1, 1].set_title("Reconstruction Penalty Over Training")
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\nStage 1 Complete:")
    print(f"  Test R²: {results_s1['final_test_r2']:.6f}")
    print(f"  Final sparsity penalty: {sparsity_y[-1]:.6f}")
    print(f"  Final reconstruction penalty: {recon_y[-1]:.6f}")

### Analyze Learned Sparsity Patterns

Examine which channels have low activation and can be safely compressed.

In [ ]:
# Analyze channel activations from Stage 1 to identify important channels
import numpy as np

# Check if Stage 1 results are available
if 'results_s1' not in globals():
    print("="*80)
    print("Stage 1 not trained yet - run Stage 1 training cell first")
    print("="*80)
else:
    logger.info("\n" + "="*80)
    logger.info("SPARSITY ANALYSIS")
    logger.info("="*80)

    important_channels = {}
    channel_importance_scores = {}

    for session_id in held_in_session_ids:
        # Get final channel activations (from last tracking point)
        if len(results_s1['channel_activations'][session_id]) > 0:
            final_epoch, final_activations = results_s1['channel_activations'][session_id][-1]
            channel_importance = final_activations.numpy()
            
            # Store importance scores
            channel_importance_scores[session_id] = channel_importance
            
            # Identify important channels (above median activation)
            median_activation = np.median(channel_importance)
            important_mask = channel_importance > median_activation
            important_idx = np.where(important_mask)[0]
            
            important_channels[session_id] = important_idx.tolist()
            
            logger.info(f"\n{session_id}:")
            logger.info(f"  Total channels: {len(channel_importance)}")
            logger.info(f"  Important channels (>median): {len(important_idx)}")
            logger.info(f"  Compression ratio: {len(important_idx)/len(channel_importance):.2%}")
            logger.info(f"  Median activation: {median_activation:.6f}")
            logger.info(f"  Max activation: {channel_importance.max():.6f}")
            logger.info(f"  Min activation: {channel_importance.min():.6f}")

    logger.info("="*80)

    # Visualize channel importance across sessions
    fig, axes = plt.subplots(len(held_in_session_ids), 1, figsize=(12, 3*len(held_in_session_ids)))
    if len(held_in_session_ids) == 1:
        axes = [axes]

    for idx, session_id in enumerate(held_in_session_ids):
        if session_id in channel_importance_scores:
            importance = channel_importance_scores[session_id]
            important_idx = important_channels[session_id]
            
            axes[idx].bar(range(len(importance)), importance, alpha=0.6, color='gray', label='All channels')
            axes[idx].bar(important_idx, importance[important_idx], alpha=0.8, color='C3', label='Important channels')
            axes[idx].axhline(np.median(importance), color='red', linestyle='--', label='Median', linewidth=2)
            axes[idx].set_xlabel("Channel Index")
            axes[idx].set_ylabel("Mean Activation")
            axes[idx].set_title(f"{session_id.split('_')[1]}_{session_id.split('_')[-1]}")
            axes[idx].legend()
            axes[idx].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("\nChannel importance scores computed. Ready for Stage 2.")

## Overall Summary

Compare all trained models (automatically detects which ones are available)

In [ ]:
# Comprehensive model comparison - automatically detects available models
import pandas as pd

# Check which models are available
models_info = {
    'Baseline (PCA AE)': ('results', 'Standard TBFM with PCA-initialized AE'),
    'Spatial Reg': ('results_spatial', 'Spatial smoothness regularization'),
    'AE Recon': ('results_ae_recon', 'AE reconstruction penalty only'),
    'Sparsity Stage 1': ('results_s1', 'Identity AE + sparsity penalties'),
}

print("="*80)
print("MODEL COMPARISON SUMMARY")
print("="*80)
print("\nAvailable Models:")

available_models = []
for name, (var_name, description) in models_info.items():
    is_available = var_name in globals()
    status = "✓" if is_available else "✗"
    print(f"  {status} {name:20s} - {description}")
    if is_available:
        available_models.append((name, var_name, globals()[var_name]))

if not available_models:
    print("\nNo models trained yet. Run training cells above.")
else:
    print("\n" + "="*80)
    print("PERFORMANCE COMPARISON")
    print("="*80)
    
    # Create comparison table
    comparison_rows = []
    for name, var_name, results in available_models:
        comparison_rows.append({
            'Model': name,
            'Test R²': f"{results['final_test_r2']:.6f}",
            'Best R²': f"{results.get('best_test_r2', results['final_test_r2']):.6f}",
        })
    
    df = pd.DataFrame(comparison_rows)
    print(df.to_string(index=False))
    
    # Calculate improvements if baseline exists
    baseline_results = next((r for n, v, r in available_models if n == 'Baseline (PCA AE)'), None)
    if baseline_results and len(available_models) > 1:
        print("\n" + "="*80)
        print("IMPROVEMENT OVER BASELINE")
        print("="*80)
        baseline_r2 = baseline_results['final_test_r2']
        
        for name, var_name, results in available_models:
            if name != 'Baseline (PCA AE)':
                r2 = results['final_test_r2']
                improvement = r2 - baseline_r2
                pct = (improvement / baseline_r2) * 100
                symbol = "+" if improvement > 0 else ""
                print(f"  {name:20s}: {symbol}{improvement:.6f} ({symbol}{pct:.2f}%)")
    
    # Per-session breakdown if multiple models
    if len(available_models) > 1:
        print("\n" + "="*80)
        print("PER-SESSION COMPARISON")
        print("="*80)
        
        # Get session list from first available model
        sessions = list(available_models[0][2]['final_test_r2s'].keys())
        
        for session_id in sessions:
            print(f"\n{session_id}:")
            for name, var_name, results in available_models:
                if session_id in results['final_test_r2s']:
                    r2 = results['final_test_r2s'][session_id]
                    print(f"  {name:20s}: {r2:.6f}")
    
    print("\n" + "="*80)
    
    # Visualize all available models
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Overall comparison bar chart
    model_names = [name for name, _, _ in available_models]
    test_r2s = [results['final_test_r2'] for _, _, results in available_models]
    
    axes[0].bar(range(len(model_names)), test_r2s, alpha=0.7)
    axes[0].set_xticks(range(len(model_names)))
    axes[0].set_xticklabels(model_names, rotation=45, ha='right')
    axes[0].set_ylabel("Test R²")
    axes[0].set_title("Overall Test R² Comparison")
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Add values on bars
    for i, v in enumerate(test_r2s):
        axes[0].text(i, v + 0.005, f"{v:.4f}", ha='center', va='bottom', fontweight='bold')
    
    # Per-session heatmap
    if len(available_models) > 1:
        sessions = list(available_models[0][2]['final_test_r2s'].keys())
        session_data = []
        for name, var_name, results in available_models:
            session_r2s = [results['final_test_r2s'].get(s, 0) for s in sessions]
            session_data.append(session_r2s)
        
        session_data = np.array(session_data).T  # Sessions x Models
        
        im = axes[1].imshow(session_data, aspect='auto', cmap='viridis')
        axes[1].set_xticks(range(len(model_names)))
        axes[1].set_xticklabels(model_names, rotation=45, ha='right')
        axes[1].set_yticks(range(len(sessions)))
        axes[1].set_yticklabels([s.split('_')[1] + '_' + s.split('_')[-1] for s in sessions])
        axes[1].set_title("Per-Session Test R²")
        
        # Add colorbar
        cbar = plt.colorbar(im, ax=axes[1])
        cbar.set_label("Test R²")
        
        # Add text annotations
        for i in range(len(sessions)):
            for j in range(len(model_names)):
                text = axes[1].text(j, i, f"{session_data[i, j]:.3f}",
                                   ha="center", va="center", color="w", fontsize=8)
    
    plt.tight_layout()
    plt.show()

## Comparison: Baseline vs Spatial Regularization

In [ ]:
logger.info("\n" + "="*80)
logger.info("COMPARISON: BASELINE vs SPATIAL")
logger.info("="*80)

# Compare overall performance
baseline_r2 = results['final_test_r2']
spatial_r2 = results_spatial['final_test_r2']

logger.info(f"\nOverall Test R²:")
logger.info(f"  Baseline: {baseline_r2:.6f}")
logger.info(f"  Spatial:  {spatial_r2:.6f}")
logger.info(f"  Improvement: {(spatial_r2 - baseline_r2):.6f} ({(spatial_r2 - baseline_r2)/baseline_r2*100:.2f}%)")

# Per-session comparison
logger.info(f"\nPer-session Test R²:")
for session_id in held_in_session_ids:
    base_r2 = results['final_test_r2s'][session_id]
    spat_r2 = results_spatial['final_test_r2s'][session_id]
    improvement = spat_r2 - base_r2
    logger.info(f"  {session_id}:")
    logger.info(f"    Baseline: {base_r2:.6f}")
    logger.info(f"    Spatial:  {spat_r2:.6f}")
    logger.info(f"    Improvement: {improvement:.6f} ({improvement/base_r2*100:.2f}%)")

logger.info("="*80)

## Comparison Visualizations

In [ ]:
# Direct comparison: Baseline vs Spatial
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Extract data
txt = [t[0] for t in results['train_losses']]
tlt = [t[1] for t in results['train_losses']]
tx = [t[0] for t in results['test_losses']]
tl = [t[1] for t in results['test_losses']]
tx_r2 = [t[0] for t in results['train_r2s']]
tr = [t[1] for t in results['train_r2s']]
te = [t[1] for t in results['test_r2s']]

txt_s = [t[0] for t in results_spatial['train_losses']]
tlt_s = [t[1] for t in results_spatial['train_losses']]
tx_s = [t[0] for t in results_spatial['test_losses']]
tl_s = [t[1] for t in results_spatial['test_losses']]
tx_r2_s = [t[0] for t in results_spatial['train_r2s']]
tr_s = [t[1] for t in results_spatial['train_r2s']]
te_s = [t[1] for t in results_spatial['test_r2s']]

# Test Loss Comparison
axes[0, 0].plot(tx, tl, label="Baseline", alpha=0.8, linewidth=2, color='C0')
axes[0, 0].plot(tx_s, tl_s, label="Spatial", alpha=0.8, linewidth=2, color='C2')
axes[0, 0].set_xlabel("Epoch")
axes[0, 0].set_ylabel("Test Loss")
axes[0, 0].set_title("Test Loss: Baseline vs Spatial")
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Test R² Comparison
axes[0, 1].plot(tx_r2, te, label="Baseline", alpha=0.8, linewidth=2, color='C0')
axes[0, 1].plot(tx_r2_s, te_s, label="Spatial", alpha=0.8, linewidth=2, color='C2')
axes[0, 1].set_xlabel("Epoch")
axes[0, 1].set_ylabel("Test R²")
axes[0, 1].set_title("Test R²: Baseline vs Spatial")
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Spatial penalty curves
smoothx = [t[0] for t in results_spatial['spatial_smooth_losses']]
smoothy = [t[1] for t in results_spatial['spatial_smooth_losses']]
decx = [t[0] for t in results_spatial['spatial_decoder_losses']]
decy = [t[1] for t in results_spatial['spatial_decoder_losses']]

axes[1, 0].plot(smoothx, smoothy, alpha=0.8, linewidth=2, color='C4')
axes[1, 0].set_xlabel("Epoch")
axes[1, 0].set_ylabel("Spatial Smoothness Penalty")
axes[1, 0].set_title("Spatial Smoothness Penalty Over Training")
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(decx, decy, alpha=0.8, linewidth=2, color='C5')
axes[1, 1].set_xlabel("Epoch")
axes[1, 1].set_ylabel("Spatial Decoder Penalty")
axes[1, 1].set_title("Spatial Decoder Penalty Over Training")
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Final comparison bar chart
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Overall comparison
overall_data = [baseline_r2, spatial_r2]
axes[0].bar(['Baseline', 'Spatial'], overall_data, color=['C0', 'C2'], alpha=0.7)
axes[0].set_ylabel("Test R²")
axes[0].set_title("Overall Test R² Comparison")
axes[0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(overall_data):
    axes[0].text(i, v + 0.001, f"{v:.5f}", ha='center', va='bottom', fontweight='bold')

# Per-session comparison
sessions = list(results['final_test_r2s'].keys())
baseline_vals = [results['final_test_r2s'][s] for s in sessions]
spatial_vals = [results_spatial['final_test_r2s'][s] for s in sessions]

x = range(len(sessions))
width = 0.35

axes[1].bar([i - width/2 for i in x], baseline_vals, width, label='Baseline', color='C0', alpha=0.7)
axes[1].bar([i + width/2 for i in x], spatial_vals, width, label='Spatial', color='C2', alpha=0.7)
axes[1].set_ylabel("Test R²")
axes[1].set_title("Per-Session Test R² Comparison")
axes[1].set_xticks(x)
axes[1].set_xticklabels([s.split('_')[1] + '_' + s.split('_')[-1] for s in sessions], rotation=45, ha='right')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("Summary Statistics")
print("="*80)
print(f"Baseline - Final Test R²: {baseline_r2:.6f}")
print(f"Spatial  - Final Test R²: {spatial_r2:.6f}")
print(f"Improvement: {(spatial_r2 - baseline_r2):.6f} ({(spatial_r2 - baseline_r2)/baseline_r2*100:.2f}%)")
print("="*80)

## Load Saved Models

To load the saved models later, use:

```python
# Load baseline model
checkpoint = torch.load(model_save_dir / "baseline_model.pt")
ms_loaded = multisession.build_from_cfg(cfg, data_train, device=DEVICE)
ms_loaded.load_state_dict(checkpoint['ms_state_dict'])
embeddings_stim_loaded = checkpoint['embeddings_stim']
results_loaded = checkpoint['results']

# Load spatial model
checkpoint_spatial = torch.load(model_save_dir / "spatial_model.pt")
cfg_spatial_loaded = OmegaConf.create(checkpoint_spatial['config'])
ms_spatial_loaded = multisession.build_from_cfg(cfg_spatial_loaded, data_train, device=DEVICE)
ms_spatial_loaded.load_state_dict(checkpoint_spatial['ms_state_dict'])
```